In [ ]:
import numpy as np
import importlib, os, datetime, itertools
from sus.protocol_designer import System, Protocol, Potential, Compound_Protocol
from sus.protocol_designer.protocol import sequential_protocol
from IPython import display
from IPython.display import HTML, Image
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation, PillowWriter
from scipy.optimize import fsolve
import matplotlib.pyplot as plt

from quick_sim import setup_sim


# import edward_tools.fq_runner as fq_runner
from edward_tools.coupled_fq_potential import coupled_flux_qubit_non_linear_approx_pot, coupled_flux_qubit_non_linear_approx_force
from edward_tools.visualization import animate_sim_flux_qubit, plotFidelityBarChart, separate_by_state_2
from edward_tools.initial_state_sampling import extra_constraint_00_and_11_only
from NAND_PARAMETERS import *
import importlib

import kyle_tools as kt
import matplotlib.pyplot as plt


from edward_tools import coupled_fq_protocol_library, cfq_runner
from edward_tools import coupled_fq_protocol_library

import edward_tools.cfq_batch_sweep as cfq_batch_sweep
import edward_tools.Analysis_tool.general_analysis_tools as general_analysis_tool
# from edward_tools.Analysis_tool.general_analysis_tools import show_phi_dc_with_time
import edward_tools.Analysis_tool.minimum_value_of_potential as minimum_value_of_potential
from edward_tools.couple_flux_qubit_metrics import fidelityEvaluation
from edward_tools import visualization

from edward_tools.pot_analysis_helper_functions import get_XYU, pot_function, find_all_critical_points_for_all_potential

coupled_fq_protocol_library = importlib.reload(coupled_fq_protocol_library)
create_system = coupled_fq_protocol_library.create_system
get_potential_shot_at_different_t = coupled_fq_protocol_library.get_potential_shot_at_different_t
get_potential_shot_at_different_t_1D = coupled_fq_protocol_library.get_potential_shot_at_different_t_1D
create_simple_protocol_parameter_dict = coupled_fq_protocol_library.create_simple_protocol_parameter_dict
coupled_fq_runner = importlib.reload(cfq_runner)
coupled_fq_protocol_library = importlib.reload(coupled_fq_protocol_library)
create_system = coupled_fq_protocol_library.create_system
get_potential_along_a_1D_cutline = coupled_fq_protocol_library.get_potential_along_a_1D_cutline
plotCutlines = coupled_fq_protocol_library.plotCutlines

In [ ]:

def plot_3_well_potential(protocol_list, left_y_range, right_y_range, bottom_left_x_range = [-4, 4], bottom_right_x_range = [-4, 4], bottom_left_y_range = [-50, 500], bottom_right_y_range = [-50, 500], contour_x_range = [-4, 4], contour_y_range = [-4, 4], beta = 2.3, d_beta = 0, phi_2_bottom_fixed = -2.1, phi_2_top_fixed = 2.0, set_same_min = False, direction = "h"):
    phi_1_array = np.linspace(-4, 4, 1000)
    params_list_used = cfqr.protocol.get_params(0)
    fig = plt.figure(figsize= [15, 7.5])
    ax_0, ax_1, ax_2, ax_3 = fig.add_subplot(2, 4, 1), fig.add_subplot(2, 4, 2), fig.add_subplot(2, 4, 3), fig.add_subplot(2, 4, 4)
    ax_4, ax_5, ax_6, ax_7 = fig.add_subplot(2, 4, 5), fig.add_subplot(2, 4, 6), fig.add_subplot(2, 4, 7), fig.add_subplot(2, 4, 8) 

    critical_point_list = []
    color_code = []

    X, Y = np.meshgrid(phi_1_array, phi_1_array)
    
    for _protocol in protocol_list:
        name     = _protocol['name']
        _protocol['beta'], _protocol['d_beta'] = beta, d_beta

        critical_points = find_all_critical_points_for_all_potential(_protocol, guess = [(-2, -2), (-2, 2), (3, -3), (2, 2)])
        critical_point_list.append(critical_points)
        params_list_used[4], params_list_used[5] = beta, beta # phi_1x
        params_list_used[8] = _protocol['phi_1x'] # phi_1x
        params_list_used[9] = _protocol['phi_2x'] # phi_2x"
        params_list_used[10] = _protocol['phi_1xdc'] # phi_1xdc
        params_list_used[11] = _protocol['phi_2xdc'] # phi_2xdc
        params_list_used[12] = _protocol['mu_12'] # mu

        fig.suptitle(f"phi_1xdc = {params_list_used[10]:.3g}, phi_2xdc = {params_list_used[11]:.3g}, phi_1x = {params_list_used[8]:.3g}, phi_2x = {params_list_used[9]:.3g}, mu = {params_list_used[12]:.3g}") 

        U = coupled_flux_qubit_non_linear_approx_pot(X, Y, _protocol['phi_1xdc'], _protocol['phi_2xdc'], params_list_used)

        if direction == "h":
            potential = coupled_flux_qubit_non_linear_approx_pot(phi_1_array, phi_2_bottom_fixed,  _protocol['phi_1xdc'], _protocol['phi_2xdc'], params_list_used)
        if direction == "v":
            potential = coupled_flux_qubit_non_linear_approx_pot(phi_2_bottom_fixed, phi_1_array,  _protocol['phi_1xdc'], _protocol['phi_2xdc'], params_list_used)
        # print(potential)
        color_code.append(_protocol['color'])
        ax_0.plot(phi_1_array, potential, label = f"{name}", c = _protocol['color'], linestyle = _protocol['linestyle'])
        ax_4.plot(phi_1_array, potential - min(potential), label = f"{name}", c = _protocol['color'], linestyle = _protocol['linestyle'])


        if direction == "h":
            potential = coupled_flux_qubit_non_linear_approx_pot(phi_1_array, phi_2_top_fixed,  _protocol['phi_1xdc'], _protocol['phi_2xdc'], params_list_used)
        if direction == "v":
            potential = coupled_flux_qubit_non_linear_approx_pot(phi_2_top_fixed, phi_1_array,  _protocol['phi_1xdc'], _protocol['phi_2xdc'], params_list_used)

        ax_2.plot(phi_1_array, potential, label = f"{name}", c = _protocol['color'], linestyle = _protocol['linestyle'])
        ax_6.plot(phi_1_array, potential - min(potential), label = f"{name}", c = _protocol['color'], linestyle = _protocol['linestyle'])

        ax_0.set_xlim(-4, 4)
        ax_0.set_ylim(left_y_range[0], left_y_range[1])
        ax_4.set_xlim(bottom_left_x_range[0], bottom_left_x_range[1])
        ax_4.set_ylim(bottom_left_y_range[0], bottom_left_y_range[1])


        ax_2.set_xlim(-4, 4)
        ax_2.set_ylim(right_y_range[0], right_y_range[1])
        ax_6.set_xlim(bottom_right_x_range[0], bottom_right_x_range[1])
        ax_6.set_ylim(bottom_right_y_range[0], bottom_right_y_range[1])
        ax_0.legend()
        

    ax_1.contourf(X, Y, U, 30)
    ax_3.contourf(X, Y, U, 30)

    if direction == "h":
        ax_1.hlines(y=phi_2_bottom_fixed, xmin = -4, xmax = 4, color = "red")
        ax_3.hlines(y=phi_2_top_fixed, xmin = -4, xmax = 4, color = "red")
        ax_5.hlines(y=phi_2_bottom_fixed, xmin = -4, xmax = 4, color = "red")
        ax_7.hlines(y=phi_2_top_fixed, xmin = -4, xmax = 4, color = "red")
    if direction == "v":
        ax_1.vlines(x=phi_2_bottom_fixed, ymin = -4, ymax = 4, color = "red")
        ax_3.vlines(x=phi_2_top_fixed, ymin = -4, ymax = 4, color = "red")
        ax_5.vlines(x=phi_2_bottom_fixed, ymin = -4, ymax = 4, color = "red")
        ax_7.vlines(x=phi_2_top_fixed, ymin = -4, ymax = 4, color = "red")

    ax_5.contourf(X, Y, U, 30)
    ax_7.contourf(X, Y, U, 30)

    
    for critical_point_info, _c in zip(critical_point_list, color_code):
        for (_x, _y) in critical_point_info['coord']:
            ax_1.scatter(_x, _y, color = _c)
            ax_3.scatter(_x, _y, color = _c)
            ax_5.scatter(_x, _y, color = _c)
            ax_7.scatter(_x, _y, color = _c)

    plt.show() 
    return critical_point_list

# control swap protocol
def createProtocol(duration, protocol, name):
    protocol_new = {
        "phi_1xdc": protocol["phi_1xdc"], "phi_2xdc": protocol["phi_2xdc"], 
        "phi_1x": protocol["phi_1x"], "phi_2x": protocol["phi_2x"], 
        "mu_12": protocol["mu_12"], "duration": duration, "name": name
    }
    return protocol_new

def setup_transcendtal_equation(phi_1xdc, beta, gamma):
    def Fcn(phi_1dc):
        return phi_1dc - beta / (2 * gamma) * np.sin(phi_1dc/2) - phi_1xdc
    return Fcn

def findBarrierHeight(_phi_1xdc, mu_factor_1, beta_1):
    protocol_2 = {"phi_1x": 0.095 * mu_factor_1, "phi_2x": 0, "mu_12": 0.0475 * mu_factor_1, "phi_1xdc": _phi_1xdc, "phi_2xdc": 0, "name": "acceleration", "color": "g", "linestyle": "-"}
    protocol_2["beta"] = beta_1
    protocol_2["d_beta"] = 0
    critical_dict = find_all_critical_points_for_all_potential(protocol_2, guess = [(-2, -2), (-2, 2), (2, -2), (2, 2), (-2,0), (0, -2), (2, 0), (0, 2)])
    critical_points = list(critical_dict.values())[0]
    critical_potential = [pot_function(protocol_2)([x, y]) for x, y in critical_points]
    delta_U17 = critical_potential[7] - critical_potential[1]
    delta_U37 = critical_potential[7] - critical_potential[3]
    return {
        "coord": critical_points, "delta_U17": delta_U17 * U0_1, "delta_U37": delta_U37 * U0_1, "phi_1xdc": _phi_1xdc
    }



In [ ]:
mapping_index = {"00": 0, "01": 1, "10": 2, "11": 3}

mapping_state_1_to_state_2_dict_NAND = {"00": ["11"], "01": ["11"], "10": ["11"], "11": ["00"]}

mapping_state_1_to_state_2_dict_in_index_form_NAND = {0: [3], 1: [3], 2: [3], 3: [0]}


In [ ]:
L = 5e-12
C = 500e-15

In [ ]:
PHI_0 = 2.067833848 * 1e-15
k_B = 1.38e-23
T = 4.2
k_BT = k_B * T
x_c = PHI_0 / (2 * np.pi)
beta_1 = 2.3

U0_1 = x_c**2 / L / k_BT

In [ ]:
protocol_parameter_for_different_beta = {
    "2.30": {
        "mu_12_factor": 0.0475, "phi_1x_factor": 0.095,
        "critical_point_guess": [(-2, -2), (-2, 2), (2, -2), (2, 2), (-2,0), (0, -2), (2, 0), (0, 2)]
    },
    "1.35": {
        "mu_12_factor": 0.0723, "phi_1x_factor": 0.095,
        "critical_point_guess": [(-1.5, -1.5), (-1.5, 1.5), (1.5, -1.5), (1.5, 1.5), (-1.5,0), (0, -1.5), (1.5, 0), (0, 1.5)]
    }
}


# intended barrier height vs effective barrier height

## difference between phi_xdc and phi_dc

In [ ]:

phi_1xdc_array = np.linspace(0, 2.1, 50)
gamma_1 = [9, 16]
beta_1 = [1.35, 2.3]
color_array = ["blue", "orange", "grey", "red"]
gamma_beta_array = [(2.3, 9), (2.3, 16), (1.35, 9), (1.35, 16)]

color_array = ["blue", "grey"]
gamma_beta_array = [(2.3, 9), (1.35, 9)]

phi_xdc_phi_dc_combination = []
for _beta, _gamma in gamma_beta_array:
    item = {
        "beta": _beta, "gamma": _gamma, "phi_xdc_vs_phi_dc": []
    }
    for _phi_1xdc in phi_1xdc_array:
        Fcn = setup_transcendtal_equation(_phi_1xdc, _beta, _gamma)
        sol = fsolve(Fcn, _phi_1xdc)[0]
        item["phi_xdc_vs_phi_dc"].append([_phi_1xdc, sol])
    phi_xdc_phi_dc_combination.append(item)

In [ ]:



fig, ax = plt.subplots(1, 1, figsize = [5, 5])
for item, color in list(zip(phi_xdc_phi_dc_combination, color_array))[0:1]:
    phi_xdc_vs_phi_dc_array = np.array(item["phi_xdc_vs_phi_dc"])
    ax.plot(phi_xdc_vs_phi_dc_array[:, 0], phi_xdc_vs_phi_dc_array[:, 1] - phi_xdc_vs_phi_dc_array[:, 0], label = r"$\beta$ = " + f"{item['beta']}" + r", $\gamma$ = " + f"{item['gamma']}", color = color)
    # ax2.plot([0, 2.1], [0, 0], linestyle= "--", color ="grey", alpha = 0.5)
    # ax2.plot(phi_xdc_vs_phi_dc_array[:, 0], phi_xdc_vs_phi_dc_array[:, 1], label = r"$\gamma$ = " + f"{item['gamma']}" + r", $\beta$ = " + f"{item['beta']}", color = color)


# ax1.set_aspect(1)
ax.legend()
ax.set_xlabel(r"$\varphi_{xdc}$", fontsize = 14)
ax.set_ylabel(r"$\varphi_{dc} - \varphi_{xdc}$", fontsize = 14)


In [ ]:


# These are in unitless percentages of the figure size. (0,0 is bottom left)
left, bottom, width, height = [0.55, 0.2, 0.2, 0.2]



In [ ]:



# mu_factor_1 = -1.1
# protocol_0_q = {"phi_1x": 0. * mu_factor_1, "phi_2x": 0, "mu_12": 0. * mu_factor_1, "phi_1xdc": 0.0, "phi_2xdc": 0, "name": "acceleration_protocol_0", "color": "g", "linestyle": "-"}

# # beta = 2.3


# params_array = [(1.90, -1), (1.945, -0.8), (1.99, -0.7), (2.01, -0.6), (2.02, -0.6), (2.04, -0.5), (2.10, -0.3)]
# selected_param = params_array[2]

# _phi_1xdc = selected_param[0]
# mu_factor_1 = selected_param[1]
# _t = 50

# mu_factor_1 = -0.5

# protocol_2 = {"phi_1x": 0.095 * mu_factor_1, "phi_2x": 0, "mu_12": 0.068 * mu_factor_1, "phi_1xdc": 0.9, "phi_2xdc": 0, "name": "acceleration", "color": "g", "linestyle": "-"}
# cutline_value = 2.0


# protocol_list_adiabatic = [
#     createProtocol(_t, protocol_2, name = "acceleration_potential"),
#     createProtocol(_t, protocol_2, name = "acceleration_potential_hold"),
#     # createProtocol(0.01, protocol_1s, name = "decceleration_potential_jump"),
#     # createProtocol(10, protocol_1s, name = "decceleration_potential_hold"),
#     {'duration': _t, "phi_1x": 0.0, "phi_2x": 0.0, "mu_12": 0.00, "phi_1xdc": 0, "phi_2xdc": 0, "name": "four_well"}
# ]  
# protocol_array = [protocol_2 ]

# # print(U0_1)

# critical_point_list = plot_3_well_potential(protocol_array, [500, 2200], [800,1200], bottom_left_y_range= [0, 100], bottom_right_y_range= [-10, 60], contour_x_range = [1.0, 2], contour_y_range = [-2.5, -1.5], 
#                                              beta = beta_1, phi_2_bottom_fixed=-cutline_value, phi_2_top_fixed= cutline_value, set_same_min=True)

## theoretical intended and effective escape rate

In [ ]:

def cal_second_derivative(_phi_1, _phi_1xdc, beta_1 = 2.3):
    return 1 - beta_1 * np.cos(_phi_1) * np.cos(_phi_1xdc/2)

def getTheoreticalErrorRate(beta_1, gamma_1, protocol_duration, _phi_1xdc_and_mu_factor_array):
    result_array = []

    if beta_1 == 2.3:
        phi_1x_ratio = 0.095
        mu_ratio = 0.0475
    if beta_1 == 1.35:
        phi_1x_ratio = 0.1
        mu_ratio = 0.0768
        

    for _phi_1xdc, mu_factor_1 in _phi_1xdc_and_mu_factor_array:
        protocol_2 = {"phi_1x": phi_1x_ratio * mu_factor_1, "phi_2x": 0, "mu_12": mu_ratio * mu_factor_1, "phi_1xdc": _phi_1xdc, "phi_2xdc": 0, "name": "acceleration", "color": "g", "linestyle": "-"}
        protocol_2["beta"] = beta_1
        protocol_2["d_beta"] = 0
        critical_dict = find_all_critical_points_for_all_potential(protocol_2, guess = [(-2, -2), (-2, 2), (2, -2), (2, 2), (-2,0), (0, -2), (2, 0), (0, 2)])
        critical_points = list(critical_dict.values())[0]
        critical_potential = [pot_function(protocol_2)([x, y]) for x, y in critical_points]
        
        delta_U17 = critical_potential[7] - critical_potential[1]
        delta_U37 = critical_potential[7] - critical_potential[3]

        Fcn = setup_transcendtal_equation(_phi_1xdc, beta_1, gamma_1)
        sol = fsolve(Fcn, _phi_1xdc)
        intended_barrier_height_info = findBarrierHeight(_phi_1xdc, mu_factor_1, beta_1)
        effective_barrier_height_info = findBarrierHeight(sol[0], mu_factor_1, beta_1)


        intended_critial_point_info = intended_barrier_height_info['coord'][1][0], intended_barrier_height_info['phi_1xdc'], intended_barrier_height_info['delta_U17']
        intended_second_derivative_array = np.array(cal_second_derivative(intended_critial_point_info[0], intended_critial_point_info[1], beta_1 = beta_1))
        intended_plasma_freq = np.sqrt(intended_second_derivative_array / (L * C))
        intended_escape_rate = intended_plasma_freq / (2 * np.pi) * np.exp(-intended_critial_point_info[2]) * protocol_duration * 1/2


        effective_critial_point_info = effective_barrier_height_info['coord'][1][0], effective_barrier_height_info['phi_1xdc'], effective_barrier_height_info['delta_U17']
        effective_second_derivative_array = np.array(cal_second_derivative(effective_critial_point_info[0], effective_critial_point_info[1], beta_1 = beta_1))
        effective_plasma_freq = np.sqrt(effective_second_derivative_array / (L * C))
        effective_escape_rate = effective_plasma_freq / (2 * np.pi) * np.exp(-effective_critial_point_info[2]) * protocol_duration * 1/2
        result_array.append({
            "phi_1xdc": intended_barrier_height_info['phi_1xdc'],
            "intended_barrier_height": intended_barrier_height_info['delta_U17'],
            "phi_1dc": effective_barrier_height_info['phi_1xdc'],
            "effective_barrier_height": effective_barrier_height_info['delta_U17'],
            "intended_escape_rate": intended_escape_rate,
            "effective_escape_rate": effective_escape_rate
        })
    return result_array


In [ ]:
beta_1 = 2.3
_phi_1xdc_and_mu_factor_array = [(2.1, -0.32), (2.04, -0.41), (2.02625, -0.5), (2.0125
, -0.5), (1.99875, -0.5), (1.985, -0.5), (1.94, -0.6), (1.90, -0.75)]


# beta_1 = 1.35
# _phi_1xdc_and_mu_factor_array = [(0.5, -0.8), (0.76, -0.65), (0.89, -0.5), (1.03, -0.4), (1.1, -0.3), (1.18, -0.25)]


gamma_1 = 9
gamma_1 = 16

N = 1e6
protocol_duration = 150 * np.sqrt(L * C)

result_array = []


for _phi_1xdc, mu_factor_1 in _phi_1xdc_and_mu_factor_array:
    protocol_2 = {"phi_1x": 0.095 * mu_factor_1, "phi_2x": 0, "mu_12": 0.0723 * mu_factor_1, "phi_1xdc": _phi_1xdc, "phi_2xdc": 0, "name": "acceleration", "color": "g", "linestyle": "-"}
    protocol_2["beta"] = beta_1
    protocol_2["d_beta"] = 0
    critical_dict = find_all_critical_points_for_all_potential(protocol_2, guess = [(-2, -2), (-2, 2), (2, -2), (2, 2), (-2,0), (0, -2), (2, 0), (0, 2)])
    critical_points = list(critical_dict.values())[0]
    critical_potential = [pot_function(protocol_2)([x, y]) for x, y in critical_points]
    
    delta_U17 = critical_potential[7] - critical_potential[1]
    delta_U37 = critical_potential[7] - critical_potential[3]

    result_array.append({
        "coord": critical_points, "delta_U17": delta_U17 * U0_1, "delta_U37": delta_U37 * U0_1, "phi_1xdc": _phi_1xdc
    })
    print(critical_points[0], critical_points[2], critical_points[5])
    print(critical_points[1], critical_points[3])

    Fcn = setup_transcendtal_equation(_phi_1xdc, beta_1, gamma_1)
    sol = fsolve(Fcn, _phi_1xdc)
    intended_barrier_height_info = findBarrierHeight(_phi_1xdc, mu_factor_1, beta_1)
    effective_barrier_height_info = findBarrierHeight(sol[0], mu_factor_1, beta_1)


    intended_critial_point_info = intended_barrier_height_info['coord'][1][0], intended_barrier_height_info['phi_1xdc'], intended_barrier_height_info['delta_U17']
    intended_second_derivative_array = np.array(cal_second_derivative(intended_critial_point_info[0], intended_critial_point_info[1]))
    intended_plasma_freq = np.sqrt(intended_second_derivative_array / (L * C))
    intended_escape_rate = intended_plasma_freq / (2 * np.pi) * np.exp(-intended_critial_point_info[2]) * protocol_duration * 1/2


    effective_critial_point_info = effective_barrier_height_info['coord'][1][0], effective_barrier_height_info['phi_1xdc'], effective_barrier_height_info['delta_U17']
    effective_second_derivative_array = np.array(cal_second_derivative(effective_critial_point_info[0], effective_critial_point_info[1]))
    effective_plasma_freq = np.sqrt(effective_second_derivative_array / (L * C))
    effective_escape_rate = effective_plasma_freq / (2 * np.pi) * np.exp(-effective_critial_point_info[2]) * protocol_duration * 1/2


    print(f"applied _phi_1xdc = {_phi_1xdc}, mu_factor_1 = {mu_factor_1}")
    print(f"{_phi_1xdc:.4g} -> {sol[0]:.4g}")
    print(f"asymmetry = {abs(delta_U17 - delta_U37):.4g}")
    print(f"intended ∆U = {intended_barrier_height_info['delta_U17']:.4g}")
    print(f"effective ∆U = {effective_barrier_height_info['delta_U17']:.4g}")
    print(f"intended error rate = {intended_escape_rate:.4g}")
    print(f"effective error rate = {effective_escape_rate:.4g}")
    print("--" * 10)



In [ ]:
import itertools

time_array = [100, 80, 60, 40, 30, 20]
params_array = [(1.90, -1), (1.945, -0.8), (1.99, -0.7), (2.04, -0.5), (2.10, -0.3)]
simParams = list(itertools.product(time_array, params_array))
result_array = []

for [_t, (_phi_1xdc, mu_factor_1)] in simParams[3:4]:
    protocol_2 = {"phi_1x": 0.095 * mu_factor_1, "phi_2x": 0, "mu_12": 0.0475 * mu_factor_1, "phi_1xdc": _phi_1xdc, "phi_2xdc": 0, "name": "acceleration", "color": "g", "linestyle": "-"}
    protocol_2["beta"] = beta_1
    protocol_2["d_beta"] = 0
    critical_dict = find_all_critical_points_for_all_potential(protocol_2, guess = [(-2, -2), (-2, 2), (2, -2), (2, 2), (-2,0), (0, -2), (2, 0), (0, 2)])
    critical_points = list(critical_dict.values())[0]

    critical_potential = [pot_function(protocol_2)([x, y]) for x, y in critical_points]
    
    delta_U17 = critical_potential[7] - critical_potential[1]
    delta_U37 = critical_potential[7] - critical_potential[3]

    result_array.append({
        "coord": critical_points, "delta_U17": delta_U17 * U0_1, "delta_U37": delta_U37 * U0_1, "phi_1xdc": _phi_1xdc
    })


In [ ]:
t_CE = 0.5e-9
L, C = 5e-12, 500e-15
beta_1 =2.3


In [ ]:
target_index = 0
target_critial_point_info = result_array[target_index]['coord'][1][0], result_array[target_index]['phi_1xdc'], result_array[target_index]['delta_U17']
# result_array[0]['coord'][3]
print(target_critial_point_info)

In [ ]:
# second_derivative_array = np.array([cal_second_derivative(_phi_1, _phi_1xdc) for _phi_1, _phi_1xdc in zip(phi_1, phi_1xdc)])
second_derivative_array = np.array(cal_second_derivative(target_critial_point_info[0], target_critial_point_info[1]))
plasma_freq = np.sqrt(second_derivative_array / (L * C))
escape_rate = plasma_freq / (2 * np.pi) * np.exp(-target_critial_point_info[2])
dwell_time = 1 / escape_rate
robustness = dwell_time / t_CE


In [ ]:
print(f"plasma_freq = {plasma_freq:.3e}")

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()



fig.suptitle(f"L = {L} H, C = {C} F, beta = {beta}")
ax1.scatter(phi_2xdc, robustness, label= "robustness")
ax1.set_ylabel("robustness")
ax1.set_xlabel(r"$\varphi_{2xdc}$")
ax1.set_yscale("log")

ax2.scatter(phi_2xdc, delta_U10, color = "red", marker = "^", label = "∆U10", alpha = 0.4)
ax2.set_ylabel("∆U (k_BT)")
# ax2.legend(bbox_to_anchor=(1.4, 1.05))

# ask matplotlib for the plotted objects and their labels
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, bbox_to_anchor=(1,1 ))

# plt.plot(phi_2xdc, robustness)
# plt.yscale("log")
# plt.ylabel("robustness")
# plt.xlabel(r"$\varphi_2$")

In [ ]:
robustness

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()



fig.suptitle(f"L = {L} H, C = {C} F, beta = {beta}")
ax1.scatter(phi_2xdc, escape_rate, label= "escape rate")
ax1.set_ylabel("escape rate (Hz)")
ax1.set_xlabel(r"$\varphi_{2xdc}$")
ax1.set_yscale("log")

ax2.scatter(phi_2xdc, delta_U10, color = "red", marker = "^", label = "∆U10")
ax2.set_ylabel("∆U (k_BT)")
# ax2.legend(bbox_to_anchor=(1.4, 1.05))

# ask matplotlib for the plotted objects and their labels
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, bbox_to_anchor=(0.55,0.99 ))

In [ ]:
simulated_work = [30.3, 29.5, 27.9, 26.8, 26, 25.5, 26.6, 29, 30.4, 31]

simulated_work_error = [0.557, 0.5965, 0.5649, 0.51, 0.469,  0.398, 0.472, 0.42, 0.346, 0.284]
simulated_error_rate = [0.132, 0, 0, 0, 0.002, 0.052, 1.54, 12.2, 26.4, 25.9]

# error rate in % (phi_2xdc)
# 0.132 (1.80 ), 0 (1.84), 0 (1.88), 0, (1.92), 0.002(1.96), 1.54 (2.05), 12.2 (2.10), 26.4 (2.15), 25.9 (2.20)


In [ ]:
plt.scatter(phi_2xdc, delta_U10, label = "∆U00", color = "red")

# plt.scatter(phi_2xdc, simulated_work, label = "simulated work") 

# plt.errorbar(x = phi_2xdc[:-1], y = simulated_work[:-1], yerr=simulated_work_error[:-1], fmt = "o")
plt.xlabel(r"$\varphi_2$")
plt.ylabel(r"energy (k_BT)")
plt.legend()

In [ ]:
corrected_phi_1dc_array

In [ ]:
mu_factor_1 = -0.5
_phi_1xdc_array = np.linspace(1.90, 2.2, 40)
corrected_phi_1dc_array = []

gamma_beta_array = [(2.3, 9)]
for _beta, _gamma in gamma_beta_array:
    for _phi_1xdc in _phi_1xdc_array:
        Fcn = setup_transcendtal_equation(_phi_1xdc, _beta, _gamma)
        sol = fsolve(Fcn, _phi_1xdc)[0]
        corrected_phi_1dc_array.append(sol)


protocol_duration = np.sqrt(L * C) * 80

result_array = []


for _phi_1xdc in _phi_1xdc_array:
    protocol_2 = {"phi_1x": 0.095 * mu_factor_1, "phi_2x": 0, "mu_12": 0.0475 * mu_factor_1, "phi_1xdc": _phi_1xdc, "phi_2xdc": 0, "name": "acceleration", "color": "g", "linestyle": "-"}
    protocol_2["beta"] = beta_1
    protocol_2["d_beta"] = 0
    critical_dict = find_all_critical_points_for_all_potential(protocol_2, guess = [(-2, -2), (-2, 2), (2, -2), (2, 2), (-2,0), (0, -2), (2, 0), (0, 2)])
    critical_points = list(critical_dict.values())[0]

    critical_potential = [pot_function(protocol_2)([x, y]) for x, y in critical_points]
    
    delta_U17 = critical_potential[7] - critical_potential[1]
    delta_U37 = critical_potential[7] - critical_potential[3]

    result_array.append({
        "coord": critical_points, "delta_U17": delta_U17 * U0_1, "delta_U37": delta_U37 * U0_1, "phi_1xdc": _phi_1xdc
    })

escape_rate_info_array = []
for item in result_array:
    # second_derivative_array = np.array([cal_second_derivative(_phi_1, _phi_1xdc) for _phi_1, _phi_1xdc in zip(phi_1, phi_1xdc)])
    target_critial_point_info = item['coord'][1][0], item['phi_1xdc'], item['delta_U17']
    second_derivative_array = np.array(cal_second_derivative(target_critial_point_info[0], target_critial_point_info[1]))
    plasma_freq = np.sqrt(second_derivative_array / (L * C))
    escape_rate = plasma_freq / (2 * np.pi) * np.exp(-target_critial_point_info[2])
    dwell_time = 1 / escape_rate
    escape_rate_info_array.append([item['delta_U17'], escape_rate, escape_rate * protocol_duration])


In [ ]:
corrected_result_array = []
for _phi_1xdc in corrected_phi_1dc_array:
    protocol_2 = {"phi_1x": 0.095 * mu_factor_1, "phi_2x": 0, "mu_12": 0.0475 * mu_factor_1, "phi_1xdc": _phi_1xdc, "phi_2xdc": 0, "name": "acceleration", "color": "g", "linestyle": "-"}
    protocol_2["beta"] = beta_1
    protocol_2["d_beta"] = 0
    critical_dict = find_all_critical_points_for_all_potential(protocol_2, guess = [(-2, -2), (-2, 2), (2, -2), (2, 2), (-2,0), (0, -2), (2, 0), (0, 2)])
    critical_points = list(critical_dict.values())[0]

    critical_potential = [pot_function(protocol_2)([x, y]) for x, y in critical_points]
    
    delta_U17 = critical_potential[7] - critical_potential[1]
    delta_U37 = critical_potential[7] - critical_potential[3]

    corrected_result_array.append({
        "coord": critical_points, "delta_U17": delta_U17 * U0_1, "delta_U37": delta_U37 * U0_1, "phi_1xdc": _phi_1xdc
    })

corrected_escape_rate_info_array = []
for item in corrected_result_array:
    # second_derivative_array = np.array([cal_second_derivative(_phi_1, _phi_1xdc) for _phi_1, _phi_1xdc in zip(phi_1, phi_1xdc)])
    target_critial_point_info = item['coord'][1][0], item['phi_1xdc'], item['delta_U17']
    second_derivative_array = np.array(cal_second_derivative(target_critial_point_info[0], target_critial_point_info[1]))
    plasma_freq = np.sqrt(second_derivative_array / (L * C))
    escape_rate = plasma_freq / (2 * np.pi) * np.exp(-target_critial_point_info[2])
    dwell_time = 1 / escape_rate
    corrected_escape_rate_info_array.append([item['delta_U17'], escape_rate, escape_rate * protocol_duration])


In [ ]:
escape_rate_info_array = np.array(escape_rate_info_array)
corrected_escape_rate_info_array = np.array(corrected_escape_rate_info_array)
N = 124_000
barrier_height = escape_rate_info_array[:, 0]
corrected_barrier_height = corrected_escape_rate_info_array[:, 0]
expected_escape_number = escape_rate_info_array[:, 1] * protocol_duration * N/2
corrected_expected_escape_number = corrected_escape_rate_info_array[:, 1] * protocol_duration * N/2

In [ ]:
# Experiment 2 (2025-07-15) (3rd data, i.e. E_B = 25.0), Experiment 2 (2025-07-12)
simulation_data_barrier = [11.3, 21.0, 25.0, 30, 40.2, 50]
simulation_data_escape_event = [31193, 1112, 60, 0, 0, 0]
simulated_work = [24.4, 22.7, 25.4, 29.4, 34.6, 55]
simulated_work_error = np.array([0.225, 0.228, 0.324, 0.366, 0.463, 0.742]) * 3



In [ ]:
plt.scatter(simulation_data_barrier, simulation_data_escape_event, label = "simulation", color = "blue")
plt.scatter(barrier_height, expected_escape_number, label = "theory", color = "orange")

plt.xlabel("$\Delta E_B (k_BT)$")
plt.ylabel(r"escape events per operation")
plt.yscale("log")
plt.title(r"t = 50 $t_c$")
xmin, xmax = plt.xlim()
# plt.hlines(y = 1e-6, xmin = xmin, xmax = xmax, linestyles = "--")
plt.legend()

In [ ]:
plt.scatter(simulation_data_barrier, simulation_data_escape_event, label = "simulation", color = "blue")
plt.scatter(barrier_height, expected_escape_number, label = "theory", color = "orange")
plt.scatter(corrected_barrier_height, corrected_expected_escape_number, label = "corrected", color = "red")
plt.xlabel("$\Delta E_B (k_BT)$")
plt.ylabel(r"escape events per operation")
plt.yscale("log")
plt.title(r"t = 50 $t_c$")
plt.xlim(10, 50)
plt.ylim(1e-15, 1e5)
xmin, xmax = plt.xlim()
plt.hlines(y = 1, xmin = xmin, xmax = xmax, linestyles = "--")
plt.legend()

In [ ]:
plt.errorbar(x = simulation_data_barrier, y = simulated_work, yerr = simulated_work_error, label = "simulation", fmt = "o")
plt.xlabel("∆E_B (k_BT)")
plt.ylabel("work")
plt.title("work done vs ∆E_B")
plt.legend()

In [ ]:
protocol_duration

In [ ]:
plt.scatter(barrier_height, 1/escape_rate_info_array[:,1] / protocol_duration)
plt.xlabel("∆E_B (k_BT)")
plt.ylabel("robustness")
plt.title("t = 80 t_c, L = 5pH, C = 500fF")
plt.yscale("log")

In [ ]:
k_BT

In [ ]:
expected_escape_number

In [ ]:
np.sqrt(L * C) 

# adiabatic escape rate

In [ ]:

# gamma = 16
parameter_info_1_35 = {
    "N": 120_000, "t": 150, "beta": 1.35, "gamma": 16
}
phi_1xdc_and_phi_1dc_array_beta_1_35_gamma_16 = np.array([(0.76, 0.776), (0.89, 0.909), (1.03, 1.05), (1.1, 1.12), (1.18, 1.20)])
intended_dU_and_effective_dU_beta_1_35_gamma_16 = np.array([(40.1, 39.2), (30, 28.9), (20.2, 19), (15, 13.8), (10.3, 9.19)])
parameter_info_2_3 = {
    "N": 128_000, "t": 120, "beta": 2.3, "gamma": 16
}


simulation_result_beta_1_35_gamma_9 = {
    "t=150": {
        "experiment_label": ["", "Experiment 1 (2025-07-30)"],
        "work_array": np.array([(25.2, 0.111), (20.2, 0.0936), (13.7, 0.0623), (11.6, 0.0519), (np.nan, np.nan)]),
        "error_array": np.array([0, 0, 5, 100, np.nan, ]),
        "error_rate_array": np.array([0, 0, 5.513e-06, 9.091e-05,  np.nan]),
        "N": np.array([1_350_000, 1_188_000, 1_161_000, 1_100_000, np.nan])
    },
    "t=75": {
        "experiment_label": ["", "Experiment 1 (2025-07-29)"],
        "work_array": np.array([(26, 0.123), (21.2, 0.153), (15.6, 0.0659), (14, 0.055)]),
        "error_array": np.array([0, 0, 0, 69]),
        "error_rate_array": np.array([0, 0, 0, 6.063e-5]),
        "N": np.array([1_142_000, 33_000, 1_137_000, 1_137_931])
    },
    "t=42": {
        "experiment_label": ["Experiment 4 (2025-07-29)"],
        "work_array": np.array([(None, None), (None, None), (None, None), (20.2, 0.173)]),
        "error_array": np.array([None, None, None, 2.000e-03]),
        "N": np.array([0, 0, 0, 789_000])
    }
}


simulation_result_beta_1_35_gamma_16 = {
    "t=150": {
        "experiment_label": ["Experiment 4 (2025-07-20)"],
        "work_array": np.array([(25.2, 0.129), (18.6, 0.0942), (14.1, 0.0681), (10.3, 0.0482), (np.nan, np.nan)]),
        "error_array": np.array([0, 0, 0, 47, np.nan]),
        "error_rate_array": np.array([0, 0, 0, 2.114e-05, np.nan]),
        "N": np.array([140_000, 140_000, 140_000, 2_271_000])
    },
    "t=75": {
        "experiment_label": ["Experiment 5 (2025-07-20)", "Experiment 1 (2025-08-31) [only the case of smallest barrier height]"],
        "work_array": np.array([(26.2, 0.13), (19.7, 0.0954), (15.8, 0.0704), (17.3, 0.0743)]),
        "error_rate_array": np.array([0, 0, 1.969e-06,  3.696e-05]),
        "error_array": np.array([0, 0, 2, 38]),
        "N": np.array([1_016_000, 1_016_000, 1_016_000, 1_028_000])
    },
    "t=42": {
        "experiment_label": ["Experiment 4 (2025-07-29) [not shown here because it was bad]", "Experiment 3 (2025-08-29)"],
        "work_array": np.array([(None, None), (None, None), (None, None), (22.5, 0.0766)]),
        "error_rate_array": np.array([0, 0, np.nan,  8.566e-05]),
        "error_array": np.array([None, None, None, 98]),
        "N": np.array([np.nan, np.nan, np.nan, 1_144_000])
    },
}

simulation_result_beta_2_3_gamma_16 = {
    "t=150": {
        "experiment_label": ["Experiment 2 (2025-07-21)"],
        "phi_1xdc_and_phi_1dc": [],
        "barrier_height": [(49.88, 35.73), (40.32, 27.13), (30.39, 18.54), (27.55, 16.16), (24.81, 13.91), (22.18, 11.8), (19.72, 9.894), (10.27, 3.253)], # (intended, effective)
       "theoretical_error_rate": [(0.0002166, 0.1814), (2.017e-08, 0.0003136), (1.782e-09, 4.865e-05), (1.321e-10, 6.166e-06), (8.756e-12, 6.771e-07), (5.216e-13, 6.483e-08), (2.751e-17, 1.326e-11), (2.04e-21, 2.618e-15)],  # (intended, effective)

        "work_array": np.array([(38.5, 0.175), (30.9, 0.132), 
                                
                                (27.2, 0.101), (26.5, 0.104), (26.0, 0.107), (25.5, 0.101), (25.0, 0.0771), (25.3, 0.0528)]) * 1.3,
        "error_rate_array": np.array([0, 0, 0, 0.000e+00, 2.700e-05, 9.216e-05, 1.700e-04, 8.155e-02]),
        "error_array": np.array([0, 0, 0, 94, 83_000])
    },
    "t=75": {
        "experiment_label": ["Experiment 1 (2025-07-23)"],
        "work_array": np.array([(47.5, 0.26), (41.9, 0.192), (38.1, 0.0809), (40.3, 0.102), (65.9, 0.11)]),
        "error_array": np.array([0, 0, 0, 4_355,  263_127]),
        "error_rate_array": np.array([0, 0, 0, 2.903e-03, 2.631e-01]),
        "N": np.array([500_000, 500_000, 0, 1_500_000, 1,500,000, 1_000_000]),
        
    }
}


simulation_result_beta_2_3_gamma_9 = {
    "experiment_label": ["Experiment 1 (2025-07-28)", "Experiment 1 (2025-10-15)", "Experiment 2 (2025-10-15)", "Experiment 3 (2025-10-15)"],
    "t=150": {
        "barrier_height": [(49.88, 25.58), (40.32, 17.99), (30.39, 10.76), (27.55, 8.852), (24.81, 7.106), (22.18, 5.528), (19.72, 4.171), (10.27, 0.3497)], # (intended, effective)
        "theoretical_error_rate": [(7.22e-5, 0.25), (6.822e-9, 0.02573), (5.94e-10, 0.007113), (4.403e-11, 0.001564), (2.919e-12, 0.0002883), (1.739e-13, 4.495e-5), (9.169e-18, 3.726e-8), (6.798e-22, 2.055e-11)],  # (intended, effective)
        "work_array": np.array([(38, 0.172), (30.9, 0.128), 
        
        (28.0, 0.0978), (27.5, 0.0999), (27.0, 0.103), (27.2, 0.0983), (27.5, 0.0686), (40.2, 0.0765)]) * 1.3,
        "error_rate_array": np.array([0, 0, 6.333e-05, 9.420e-4, 5.976e-3, 1.885e-02, 2.671e-2, 2.659e-01]),
        "error_array": np.array([0, 0, 0, 942, 5_976, 18_850, 26_706, 265_890])
    },
    "t=75": {
        "work_array": np.array([]),
        "error_array": np.array([])
    }
}


# gamma = 16

phi_1xdc_and_phi_1dc_array_beta_2_3_gamma_16 = np.array([(1.9, 1.96), (1.94, 2), (1.985, 2.046),  (1.999, 2.06), (2.013, 2.074), (2.026, 2.088), (2.04, 2.102), (2.1, 2.163)])
intended_dU_and_effective_dU_beta_2_3_gamma_16 = np.array([(49.88, 25.58), (40.32, 17.99), (30.39, 10.76), (27.55, 8.852), (24.81, 7.106), (22.18, 5.528), (19.72, 4.171), (10.27, 0.3497)])


# gamma = 9
phi_1xdc_and_phi_1dc_array_beta_1_35_gamma_9 = np.array([(0.76, 0.789), (0.89, 0.934), (1.03, 1.07), (1.1, 1.14), (1.18, 1.22)])
intended_dU_and_effective_dU_beta_1_35_gamma_9 = np.array([(40.1, 38.4), (30, 28), (20.2, 18.1), (15, 12.9), (10.3, 8.33)])

phi_1xdc_and_phi_1dc_array_beta_2_3_gamma_9 = np.array([(1.9, 2.01), (1.94, 2.05), (1.985, 2.1), (2.04, 2.15), (2.1, 2.21)])
intended_dU_and_effective_dU_beta_2_3_gamma_9 = np.array([(49.9, 25.6), (40.3, 18), (30.4, 10.8), (27.55, 8.852), (24.81, 7.106), (22.18, 5.528), (19.72, 4.171), (10.27, 0.3497)])

In [ ]:
opacity = 0.3
opacity = 1
show_all_data = True

fig, ax = plt.subplots(3, 1, figsize = [5, 15])
ax[0].scatter(intended_dU_and_effective_dU_beta_2_3_gamma_9[:, 0], intended_dU_and_effective_dU_beta_2_3_gamma_9[:, 1], label=r"$\beta= 2.3$, $\gamma = 9$", color = "blue", alpha = opacity)
ax[0].scatter(intended_dU_and_effective_dU_beta_2_3_gamma_16[:, 0], intended_dU_and_effective_dU_beta_2_3_gamma_16[:, 1], label=r"$\beta= 2.3$, $\gamma = 16$", color = "orange", alpha = opacity)
if opacity < 1 or show_all_data:
    ax[0].scatter(intended_dU_and_effective_dU_beta_1_35_gamma_9[:, 0], intended_dU_and_effective_dU_beta_1_35_gamma_9[:, 1], label=r"$\beta= 1.35$, $\gamma = 9$", marker="s", color = "grey")
    ax[0].scatter(intended_dU_and_effective_dU_beta_1_35_gamma_16[:, 0], intended_dU_and_effective_dU_beta_1_35_gamma_16[:, 1], label=r"$\beta= 1.35$, $\gamma = 16$", marker="s", color = "red")

# ax[0].set_xlabel(r"intended $\Delta E_B$ ($k_BT$)")
ax[0].set_ylabel(r"effective $\Delta E_B$ ($k_BT$)")
ax[0].set_aspect(1)
ax[0].set_xlim(0, 55)
ax[0].set_ylim(0, 55)
ax[0].plot([0, 55], [0, 55], linestyle = "--", alpha = 0.5)
ax[0].legend(fontsize = 15, loc='best')


# ax[1].errorbar(intended_dU_and_effective_dU_beta_1_35_gamma_16[:-1, 0], work_array_beta_1_35_gamma_16[:, 0], yerr = work_array_beta_1_35_gamma_16[:, 1], label=r"$\beta= 1.35$, $\gamma = 16$", fmt="o")


# label=r"$\beta= 2.3$, $\gamma = 16$"
# ax[1].errorbar(intended_dU_and_effective_dU_beta_1_35_gamma_9[:, 0], simulation_result_beta_1_35_gamma_9["t=150"]["work_array"][:, 0] * 1.3, yerr = simulation_result_beta_1_35_gamma_9["t=150"]["work_array"][:, 1], label=r"$\beta= 1.35$, $\gamma = 9$", fmt="s", color = "grey", alpha = 0.3)


ax[1].errorbar(intended_dU_and_effective_dU_beta_2_3_gamma_9[:, 0], simulation_result_beta_2_3_gamma_9["t=150"]["work_array"][:, 0], yerr = simulation_result_beta_2_3_gamma_9["t=150"]["work_array"][:, 1], label=r"$\beta= 2.3$, $\gamma = 9$", fmt="o", color = "blue", alpha = opacity)
ax[1].errorbar(intended_dU_and_effective_dU_beta_2_3_gamma_16[:, 0], simulation_result_beta_2_3_gamma_16["t=150"]["work_array"][:, 0], yerr = simulation_result_beta_2_3_gamma_16["t=150"]["work_array"][:, 1], label=r"$\beta= 2.3$, $\gamma = 16$", fmt="o", color = "orange", alpha = opacity)

if opacity < 1 or show_all_data:
    ax[1].errorbar(intended_dU_and_effective_dU_beta_1_35_gamma_9[:, 0], simulation_result_beta_1_35_gamma_9["t=150"]["work_array"][:, 0], yerr = simulation_result_beta_1_35_gamma_9["t=150"]["work_array"][:, 1], label=r"$\beta= 1.35$, $\gamma = 9$", fmt="s", color = "grey")
    ax[1].errorbar(intended_dU_and_effective_dU_beta_1_35_gamma_16[:, 0], simulation_result_beta_1_35_gamma_16["t=150"]["work_array"][:, 0], yerr = simulation_result_beta_1_35_gamma_16["t=150"]["work_array"][:, 1], label=r"$\beta= 1.35$, $\gamma = 16$", fmt="s", color = "red")


# ax[1].errorbar(intended_dU_and_effective_dU_beta_2_3_gamma_16[:, 0], simulation_result_beta_2_3_gamma_16["t=75"]["work_array"][:, 0], yerr = simulation_result_beta_2_3_gamma_16["t=75"]["work_array"][:, 1], label=r"t=75", fmt="o")
ax[1].set_title(r"t = 150$t_c$")

# ax[1].legend()
# ax[1].set_xlabel(r"intended $\Delta E_B$ ($k_BT$)")
ax[1].set_ylabel(r"work ($k_BT$)")


ax[2].set_title("N = 1,000,000")
# ax[2].scatter(intended_dU_and_effective_dU_beta_1_35_gamma_16[:-1, 0], error_rate_beta_1_35_gamma_16, label=r"$\beta= 1.35$, $\gamma = 16$")
ax[2].scatter(intended_dU_and_effective_dU_beta_2_3_gamma_9[:, 0], simulation_result_beta_2_3_gamma_9["t=150"]["error_rate_array"], label=r"$\beta= 2.3$, $\gamma = 9$", marker="o", color = "blue", alpha = opacity)
ax[2].scatter(intended_dU_and_effective_dU_beta_2_3_gamma_16[:, 0], simulation_result_beta_2_3_gamma_16["t=150"]["error_rate_array"], label=r"$\beta= 2.3$, $\gamma = 16$", marker="o", color = "orange", alpha = opacity)
ax[2].scatter(intended_dU_and_effective_dU_beta_1_35_gamma_9[:, 0], simulation_result_beta_1_35_gamma_9["t=150"]["error_rate_array"], label=r"$\beta= 1.35$, $\gamma = 9$", marker="s", color = "grey")
ax[2].scatter(intended_dU_and_effective_dU_beta_1_35_gamma_16[:, 0], simulation_result_beta_1_35_gamma_16["t=150"]["error_rate_array"], label=r"$\beta= 1.35$, $\gamma = 16$", marker="s", color = "red")


# add theoretical escape rate

_phi_1xdc_and_mu_factor_array_beta_2_3 = np.vstack([np.linspace(1.90, 2.2, 40), np.linspace(-0.75, -0.32, 40)]).T
# _phi_1xdc_and_mu_factor_array_beta_1_35 = np.vstack([np.linspace(0.76, 1.18, 40), np.linspace(-0.7, -0.25, 40)]).T

_phi_1xdc_and_mu_factor_array_beta_2_3 = np.vstack([np.linspace(1.90, 2.1, 40), np.linspace(-0.75, -0.32, 40)]).T
_phi_1xdc_and_mu_factor_array_beta_1_35 = np.vstack([np.linspace(0.76, 1.18, 40), np.linspace(-0.7, -0.25, 40)]).T
protocol_duration = 150 * np.sqrt(L * C)

result_array_beta_2_3_gamma_16 = getTheoreticalErrorRate(beta_1 = 2.3, gamma_1 = 16, protocol_duration = protocol_duration, _phi_1xdc_and_mu_factor_array = _phi_1xdc_and_mu_factor_array_beta_2_3)
result_array_beta_2_3_gamma_9 = getTheoreticalErrorRate(beta_1 = 2.3, gamma_1 = 9, protocol_duration = protocol_duration, _phi_1xdc_and_mu_factor_array = _phi_1xdc_and_mu_factor_array_beta_2_3)

result_array_beta_1_35_gamma_16 = getTheoreticalErrorRate(beta_1 = 1.35, gamma_1 = 16, protocol_duration = protocol_duration * 5, _phi_1xdc_and_mu_factor_array = _phi_1xdc_and_mu_factor_array_beta_1_35)
result_array_beta_1_35_gamma_9 = getTheoreticalErrorRate(beta_1 = 1.35, gamma_1 = 9, protocol_duration = protocol_duration * 10, _phi_1xdc_and_mu_factor_array = _phi_1xdc_and_mu_factor_array_beta_1_35)



theoretical_intended_barrier_height_beta_2_3_gamma_9 = [item["intended_barrier_height"] for item in result_array_beta_2_3_gamma_9]
theoretical_intended_escape_rate_beta_2_3_gamma_9 = [item["intended_escape_rate"] for item in result_array_beta_2_3_gamma_9]
theoretical_effective_escape_rate_beta_2_3_gamma_9 = [item["effective_escape_rate"] for item in result_array_beta_2_3_gamma_9]

theoretical_intended_barrier_height_beta_2_3_gamma_16 = [item["intended_barrier_height"] for item in result_array_beta_2_3_gamma_16]
theoretical_intended_escape_rate_beta_2_3_gamma_16 = [item["intended_escape_rate"] for item in result_array_beta_2_3_gamma_16]
theoretical_effective_escape_rate_beta_2_3_gamma_16 = [item["effective_escape_rate"] for item in result_array_beta_2_3_gamma_16]

theoretical_intended_barrier_height_beta_1_35_gamma_16 = [item["intended_barrier_height"] for item in result_array_beta_1_35_gamma_16]
theoretical_intended_escape_rate_beta_1_35_gamma_16 = [item["intended_escape_rate"] for item in result_array_beta_1_35_gamma_16]
theoretical_effective_escape_rate_beta_1_35_gamma_16 = [item["effective_escape_rate"] for item in result_array_beta_1_35_gamma_16]

theoretical_intended_barrier_height_beta_1_35_gamma_9 = [item["intended_barrier_height"] for item in result_array_beta_1_35_gamma_9]
theoretical_intended_escape_rate_beta_1_35_gamma_9 = [item["intended_escape_rate"] for item in result_array_beta_1_35_gamma_9]
theoretical_effective_escape_rate_beta_1_35_gamma_9 = [item["effective_escape_rate"] for item in result_array_beta_1_35_gamma_9]


ax[2].plot(theoretical_intended_barrier_height_beta_2_3_gamma_16, theoretical_effective_escape_rate_beta_2_3_gamma_16, color = "orange", linestyle = "--")
ax[2].plot(theoretical_intended_barrier_height_beta_2_3_gamma_9, theoretical_effective_escape_rate_beta_2_3_gamma_9, color = "blue", linestyle = "--")

ax[2].plot(theoretical_intended_barrier_height_beta_1_35_gamma_16, theoretical_effective_escape_rate_beta_1_35_gamma_16, color = "red", linestyle = "--")
ax[2].plot(theoretical_intended_barrier_height_beta_1_35_gamma_9, theoretical_effective_escape_rate_beta_1_35_gamma_9, color = "grey", linestyle = "--")


ax[2].axhline(1e-6, color = "black", linestyle = "--")

# ax[2].legend()
ax[2].set_yscale("log")
ax[2].set_xlabel(r"intended $\Delta E_B$ ($k_BT$)")
ax[2].set_ylabel(r"error rate")



In [ ]:
plt.plot(theoretical_intended_escape_rate_beta_1_35_gamma_9, theoretical_effective_escape_rate_beta_1_35_gamma_9, color = "grey")
plt.plot(theoretical_intended_escape_rate_beta_1_35_gamma_16, theoretical_effective_escape_rate_beta_1_35_gamma_16, color = "red")
plt.plot(theoretical_intended_escape_rate_beta_2_3_gamma_16, theoretical_effective_escape_rate_beta_2_3_gamma_16, color = "orange")
plt.plot(theoretical_intended_escape_rate_beta_2_3_gamma_9, theoretical_effective_escape_rate_beta_2_3_gamma_9, color = "blue")
plt.plot([1e-15, 1e-1], [1e-15, 1e-1], color = "black", linestyle = "--")
plt.xlabel("intended escape rate")
plt.ylabel("effective escape rate")
plt.xlim(1e-15, 1e-4)
plt.xscale("log")
plt.yscale("log")

In [ ]:

phi_1xdc_array = np.linspace(0, 2.1, 50)
gamma_1 = [9, 16]
beta_1 = [1.35, 2.3]
# gamma_beta_array = [(2.3, 5), (2.3, 9), (2.3, 16), (1.35, 5), (1.35, 9), (1.35, 16)]
# color_array = ["pink", "blue", "orange", "grey", "red", "black"]

gamma_beta_array = [(2.3, 9), (2.3, 16), (1.35, 9), (1.35, 16)]
color_array = ["blue", "orange", "grey", "red"]
opacity = 1

phi_xdc_phi_dc_combination = []
for _beta, _gamma in gamma_beta_array:
    item = {
        "beta": _beta, "gamma": _gamma, "phi_xdc_vs_phi_dc": []
    }
    for _phi_1xdc in phi_1xdc_array:
        Fcn = setup_transcendtal_equation(_phi_1xdc, _beta, _gamma)
        sol = fsolve(Fcn, _phi_1xdc)[0]
        item["phi_xdc_vs_phi_dc"].append([_phi_1xdc, sol])
    phi_xdc_phi_dc_combination.append(item)

print(phi_xdc_phi_dc_combination)


In [ ]:
fig, ax = plt.subplots(2, 1, figsize = [5, 10])



# These are in unitless percentages of the figure size. (0,0 is bottom left)
left, bottom, width, height = [0.6, 0.1, 0.3, 0.3]
# ax2 = ax[0].inset_axes([left, bottom, width, height])

for item, color in list(zip(phi_xdc_phi_dc_combination, color_array)):
    phi_xdc_vs_phi_dc_array = np.array(item["phi_xdc_vs_phi_dc"])
    # ax[0].plot([0, 2.1], [0, 2.1], linestyle= "--", color ="grey", alpha = 0.5)
    ax[0].plot(phi_xdc_vs_phi_dc_array[:, 0], phi_xdc_vs_phi_dc_array[:, 1] - phi_xdc_vs_phi_dc_array[:, 0], label = r"$\gamma$ = " + f"{item['gamma']}" + r", $\beta$ = " + f"{item['beta']}", color = color)
    # ax2.plot([0, 2.1], [0, 2.1], linestyle= "--", color ="grey", alpha = 0.5)
    # ax2.plot(phi_xdc_vs_phi_dc_array[:, 0], phi_xdc_vs_phi_dc_array[:, 1], label = r"$\gamma$ = " + f"{item['gamma']}" + r", $\beta$ = " + f"{item['beta']}", color = color)
    # ax2.set_xlim(1.90, 2.0)
    # ax2.set_ylim(1.90, 2.1)


# ax[0].set_aspect(1)
ax[0].legend()
ax[0].set_xlabel(r"$\varphi_{xdc}$", fontsize = 14)
ax[0].set_ylabel(r"$\varphi_{dc} - \varphi_{xdc}$", fontsize = 14)


# ax[1].scatter(intended_dU_and_effective_dU_beta_2_3_gamma_9[:, 0], intended_dU_and_effective_dU_beta_2_3_gamma_9[:, 1], label=r"$\beta= 2.3$, $\gamma = 9$", color = "blue", alpha = opacity)
# ax[1].scatter(intended_dU_and_effective_dU_beta_2_3_gamma_16[:, 0], intended_dU_and_effective_dU_beta_2_3_gamma_16[:, 1], label=r"$\beta= 2.3$, $\gamma = 16$", color = "orange", alpha = opacity)
# ax[1].scatter(intended_dU_and_effective_dU_beta_1_35_gamma_9[:, 0], intended_dU_and_effective_dU_beta_1_35_gamma_9[:, 1], label=r"$\beta= 1.35$, $\gamma = 9$", marker="s", color = "grey")
# ax[1].scatter(intended_dU_and_effective_dU_beta_1_35_gamma_16[:, 0], intended_dU_and_effective_dU_beta_1_35_gamma_16[:, 1], label=r"$\beta= 1.35$, $\gamma = 16$", marker="s", color = "red")
# ax[1].set_xlabel(r"intended $\Delta E_B$ ($k_BT$)")
# ax[1].set_ylabel(r"effective $\Delta E_B$ ($k_BT$)")
# ax[1].set_aspect(1)
# ax[1].set_xlim(0, 55)
# ax[1].set_ylim(0, 55)
# ax[1].plot([0, 55], [0, 55], linestyle = "--", alpha = 0.5)

ax[1].plot(theoretical_intended_escape_rate_beta_1_35_gamma_9, theoretical_effective_escape_rate_beta_1_35_gamma_9, color = "grey")
ax[1].plot(theoretical_intended_escape_rate_beta_1_35_gamma_16, theoretical_effective_escape_rate_beta_1_35_gamma_16, color = "red")
ax[1].plot(theoretical_intended_escape_rate_beta_2_3_gamma_16, theoretical_effective_escape_rate_beta_2_3_gamma_16, color = "orange")
ax[1].plot(theoretical_intended_escape_rate_beta_2_3_gamma_9, theoretical_effective_escape_rate_beta_2_3_gamma_9, color = "blue")
ax[1].plot([1e-15, 1e-1], [1e-15, 1e-1], color = "black", linestyle = "--")
ax[1].set_xlabel("intended escape rate", fontsize = 14)
ax[1].set_ylabel("effective escape rate", fontsize = 14)
ax[1].set_xlim(1e-15, 1e-4)
ax[1].set_xscale("log")
ax[1].set_yscale("log")



In [ ]:
intended_dU_and_effective_dU_beta_1_35_gamma_16

In [ ]:
simulation_result_beta_1_35_gamma_16["t=150"]["work_array"]

In [ ]:
# beta = 1.35 and gamma = 16, t = [42, 75, 150]
time_array = [42, 75, 150]

work_done_array = [
    simulation_result_beta_1_35_gamma_16["t=42"]["work_array"][3][0],
    simulation_result_beta_1_35_gamma_16["t=75"]["work_array"][3][0],
    simulation_result_beta_1_35_gamma_16["t=150"]["work_array"][3][0]
]

error_rate = [
    simulation_result_beta_1_35_gamma_16["t=42"]["error_rate_array"][3],
    simulation_result_beta_1_35_gamma_16["t=75"]["error_rate_array"][3],
    simulation_result_beta_1_35_gamma_16["t=150"]["error_rate_array"][3]
]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = [5, 5])
ax.scatter(time_array, work_done_array)
ax2 = ax.twinx()
ax2.set_yscale("log")


ax2.scatter(time_array, error_rate, color = "red", marker="^")


# erasure-flip escape rate

In [ ]:
harmonic_beta_1_35_gamma_5 = {
    "experiment_label": ["Experiment 1 (2025-08-26)"],
    "protocol_time": [(9, 4, 4, 9, 25)],
    "work_array": np.array([26.1, 0.0353]),
    "error_array": np.array([0]),
     "error_rate_array": np.array([0]),
     "N": np.array([964_000])
}



In [ ]:
gamma_array = [5, 9, 16]
work_cost_array = [26.1, 31.6, 42]

plt.scatter(gamma_array, work_cost_array)
plt.xlabel(r"$\gamma$")
plt.ylabel(r"W ($k_BT$)")


# oscillation graph

In [ ]:
def setup_transcendtal_equation(phi_1xdc, beta, gamma):
    def Fcn(phi_1dc):
        return phi_1dc - beta / (2 * gamma) * np.sin(phi_1dc/2) - phi_1xdc
    return Fcn


phi_1xdc_array = [2.04]
gamma_1 = [9, 16]
beta_1 = [1.35, 2.3]
gamma_beta_array = [(2.3, 9)]
phi_xdc_phi_dc_combination = []
for _beta, _gamma in gamma_beta_array:
    item = {
        "beta": _beta, "gamma": _gamma, "phi_xdc_vs_phi_dc": []
    }
    for _phi_1xdc in phi_1xdc_array:
        Fcn = setup_transcendtal_equation(_phi_1xdc, _beta, _gamma)
        sol = fsolve(Fcn, _phi_1xdc)[0]
        item["phi_xdc_vs_phi_dc"].append([_phi_1xdc, sol])
    phi_xdc_phi_dc_combination.append(item)



In [ ]:
phi_xdc_phi_dc_combination
max_phi_dc = 2.1564740119468704
amplitude = max_phi_dc - phi_xdc_phi_dc_combination[0]['phi_xdc_vs_phi_dc'][0][1]
min_phi_dc = phi_xdc_phi_dc_combination[0]['phi_xdc_vs_phi_dc'][0][1] - amplitude

# Time analysis

In [ ]:
beta_1_35_gamma_9_speed_data = {
   "experiment_array": ["Experiment 3 (2025-07-29)", "Experiment 1 (2025-07-29)", "Experiment 1 (2025-07-30"],
  "duration": [42, 75, 150],
  "work": np.array([(20.2, 0.173), (13.9, 0.1235), (11.6, 0.152)]),
  "error": [2.723e-04, 4.274e-05, 2e-05]
}

beta_1_35_gamma_16_speed_data = {
   "experiment_array": ["Experiment 4 (2025-07-29)", None, "Experiment 2 (2025-07-30)"],
  "duration": [42, 75, 150],
  "work": np.array([(19.9, 0.0875), (np.nan, np.nan), (11.8, 0.198)]),
  "error": [1.642e-4, np.nan, 1.158e-05]
}


In [ ]:

fig, ax = plt.subplots(1, 2, figsize = [10, 5])
ax[0].errorbar(beta_1_35_gamma_9_speed_data['duration'], y = beta_1_35_gamma_9_speed_data['work'][:, 0], yerr = beta_1_35_gamma_9_speed_data['work'][:, 1], label = r"$\beta = 1.35, \gamma = 9$", fmt="s", color = "grey")
# ax[0].errorbar(beta_1_35_gamma_16_speed_data['duration'], y = beta_1_35_gamma_16_speed_data['work'][:, 0], yerr = beta_1_35_gamma_16_speed_data['work'][:, 1], label = r"$\beta = 1.35, \gamma = 16$", fmt="s", color = "red")
ax[0].set_xlabel(r"duration ($t_c$)")
ax[0].set_ylabel(r"work $(k_BT)$")
ax[0].legend()


ax[1].scatter(beta_1_35_gamma_9_speed_data['duration'], beta_1_35_gamma_9_speed_data['error'], label = r"$\beta = 1.35, \gamma = 9$", marker="s", color = "grey")
# ax[1].scatter(beta_1_35_gamma_16_speed_data['duration'], beta_1_35_gamma_16_speed_data['error'], label = r"$\beta = 1.35, \gamma = 16$", marker="s", color = "red")
ax[1].set_xlim(35, 160)
ax[1].set_xlabel(r"duration ($t_c$)")
ax[1].set_ylabel(r"error probability")
ax[1].set_yscale(r"log")
ax[1].set_ylim(1e-5, 1e-3)




# ax[0].set_ylim(10, 25)

# parameters

### circuit parameters

In [ ]:
opacity = 0.3
opacity = 1
show_all_data = True

fig, ax = plt.subplots(2, 1, figsize = [4, 8])



ax[0].errorbar(intended_dU_and_effective_dU_beta_2_3_gamma_9[:, 0], simulation_result_beta_2_3_gamma_9["t=150"]["work_array"][:, 0], yerr = simulation_result_beta_2_3_gamma_9["t=150"]["work_array"][:, 1], label=r"$\beta= 2.3$, $\gamma = 9$", fmt="o", color = "blue", alpha = opacity)
ax[0].errorbar(intended_dU_and_effective_dU_beta_2_3_gamma_16[:, 0], simulation_result_beta_2_3_gamma_16["t=150"]["work_array"][:, 0], yerr = simulation_result_beta_2_3_gamma_16["t=150"]["work_array"][:, 1], label=r"$\beta= 2.3$, $\gamma = 16$", fmt="o", color = "orange", alpha = opacity)
ax[0].errorbar(intended_dU_and_effective_dU_beta_1_35_gamma_9[:, 0], simulation_result_beta_1_35_gamma_9["t=150"]["work_array"][:, 0], yerr = simulation_result_beta_1_35_gamma_9["t=150"]["work_array"][:, 1], label=r"$\beta= 1.35$, $\gamma = 9$", fmt="s", color = "grey")
ax[0].errorbar(intended_dU_and_effective_dU_beta_1_35_gamma_16[:, 0], simulation_result_beta_1_35_gamma_16["t=150"]["work_array"][:, 0], yerr = simulation_result_beta_1_35_gamma_16["t=150"]["work_array"][:, 1], label=r"$\beta= 1.35$, $\gamma = 16$", fmt="s", color = "red")


ax[0].set_title(r"t = 150$t_c$")
ax[0].set_ylabel(r"work ($k_BT$)", fontsize = 13)



ax[1].set_title("N = 1,000,000")
ax[1].scatter(intended_dU_and_effective_dU_beta_2_3_gamma_9[:, 0], simulation_result_beta_2_3_gamma_9["t=150"]["error_rate_array"], label=r"$\beta= 2.3$, $\gamma = 9$", marker="o", color = "blue", alpha = opacity)
ax[1].scatter(intended_dU_and_effective_dU_beta_2_3_gamma_16[:, 0], simulation_result_beta_2_3_gamma_16["t=150"]["error_rate_array"], label=r"$\beta= 2.3$, $\gamma = 16$", marker="o", color = "orange", alpha = opacity)
ax[1].scatter(intended_dU_and_effective_dU_beta_1_35_gamma_9[:, 0], simulation_result_beta_1_35_gamma_9["t=150"]["error_rate_array"], label=r"$\beta= 1.35$, $\gamma = 9$", marker="s", color = "grey")
ax[1].scatter(intended_dU_and_effective_dU_beta_1_35_gamma_16[:, 0], simulation_result_beta_1_35_gamma_16["t=150"]["error_rate_array"], label=r"$\beta= 1.35$, $\gamma = 16$", marker="s", color = "red")


# add theoretical escape rate

_phi_1xdc_and_mu_factor_array_beta_2_3 = np.vstack([np.linspace(1.90, 2.2, 40), np.linspace(-0.75, -0.32, 40)]).T
# _phi_1xdc_and_mu_factor_array_beta_1_35 = np.vstack([np.linspace(0.76, 1.18, 40), np.linspace(-0.7, -0.25, 40)]).T

_phi_1xdc_and_mu_factor_array_beta_2_3 = np.vstack([np.linspace(1.90, 2.1, 40), np.linspace(-0.75, -0.32, 40)]).T
_phi_1xdc_and_mu_factor_array_beta_1_35 = np.vstack([np.linspace(0.76, 1.18, 40), np.linspace(-0.7, -0.25, 40)]).T
protocol_duration = 150 * np.sqrt(L * C)

result_array_beta_2_3_gamma_16 = getTheoreticalErrorRate(beta_1 = 2.3, gamma_1 = 16, protocol_duration = protocol_duration, _phi_1xdc_and_mu_factor_array = _phi_1xdc_and_mu_factor_array_beta_2_3)
result_array_beta_2_3_gamma_9 = getTheoreticalErrorRate(beta_1 = 2.3, gamma_1 = 9, protocol_duration = protocol_duration, _phi_1xdc_and_mu_factor_array = _phi_1xdc_and_mu_factor_array_beta_2_3)

result_array_beta_1_35_gamma_16 = getTheoreticalErrorRate(beta_1 = 1.35, gamma_1 = 16, protocol_duration = protocol_duration * 5, _phi_1xdc_and_mu_factor_array = _phi_1xdc_and_mu_factor_array_beta_1_35)
result_array_beta_1_35_gamma_9 = getTheoreticalErrorRate(beta_1 = 1.35, gamma_1 = 9, protocol_duration = protocol_duration * 10, _phi_1xdc_and_mu_factor_array = _phi_1xdc_and_mu_factor_array_beta_1_35)



theoretical_intended_barrier_height_beta_2_3_gamma_9 = [item["intended_barrier_height"] for item in result_array_beta_2_3_gamma_9]
theoretical_intended_escape_rate_beta_2_3_gamma_9 = [item["intended_escape_rate"] for item in result_array_beta_2_3_gamma_9]
theoretical_effective_escape_rate_beta_2_3_gamma_9 = [item["effective_escape_rate"] for item in result_array_beta_2_3_gamma_9]

theoretical_intended_barrier_height_beta_2_3_gamma_16 = [item["intended_barrier_height"] for item in result_array_beta_2_3_gamma_16]
theoretical_intended_escape_rate_beta_2_3_gamma_16 = [item["intended_escape_rate"] for item in result_array_beta_2_3_gamma_16]
theoretical_effective_escape_rate_beta_2_3_gamma_16 = [item["effective_escape_rate"] for item in result_array_beta_2_3_gamma_16]

theoretical_intended_barrier_height_beta_1_35_gamma_16 = [item["intended_barrier_height"] for item in result_array_beta_1_35_gamma_16]
theoretical_intended_escape_rate_beta_1_35_gamma_16 = [item["intended_escape_rate"] for item in result_array_beta_1_35_gamma_16]
theoretical_effective_escape_rate_beta_1_35_gamma_16 = [item["effective_escape_rate"] for item in result_array_beta_1_35_gamma_16]

theoretical_intended_barrier_height_beta_1_35_gamma_9 = [item["intended_barrier_height"] for item in result_array_beta_1_35_gamma_9]
theoretical_intended_escape_rate_beta_1_35_gamma_9 = [item["intended_escape_rate"] for item in result_array_beta_1_35_gamma_9]
theoretical_effective_escape_rate_beta_1_35_gamma_9 = [item["effective_escape_rate"] for item in result_array_beta_1_35_gamma_9]


ax[1].plot(theoretical_intended_barrier_height_beta_2_3_gamma_16, theoretical_effective_escape_rate_beta_2_3_gamma_16, color = "orange", linestyle = "--")
ax[1].plot(theoretical_intended_barrier_height_beta_2_3_gamma_9, theoretical_effective_escape_rate_beta_2_3_gamma_9, color = "blue", linestyle = "--")

ax[1].plot(theoretical_intended_barrier_height_beta_1_35_gamma_16, theoretical_effective_escape_rate_beta_1_35_gamma_16, color = "red", linestyle = "--")
ax[1].plot(theoretical_intended_barrier_height_beta_1_35_gamma_9, theoretical_effective_escape_rate_beta_1_35_gamma_9, color = "grey", linestyle = "--")


ax[1].axhline(1e-6, color = "black", linestyle = "--")

# ax[2].legend()
ax[1].set_yscale("log")
ax[1].set_xlabel(r"intended $\Delta E_B$ ($k_BT$)", fontsize = 13)
ax[1].set_ylabel(r"error rate", fontsize = 13)
ax[1].legend(fontsize = 11, loc = "lower left")


In [ ]:
print(f"L_1 = {L_1 * 1e12:.3g}pH, T = {T}K")
print(f"freq = {freq / 1e9:.3g}GHz")

### parameter setting

In [ ]:
"""
# step 0: modify parameters
- All the parameters are stored in a separate file PARAMETER_INPUT
- You can override some of the parameters here.
"""
params = {}
params['N'] = 1000
params['dt'] = 1/100
params['lambda'] = 1
params['beta'] = 1
params['sim_params'] = [_lambda, _theta, _eta]
params['target_work'] = None
params['applyOffset'] = False
params['monitor_work_dist_in_whole_process'] = True # To monitor the work process
params['comment'] = "Experiment 8 (2024/3/17): 4 well, with no compensation for asym, 1/5000"
params['capacitance'] = [C_1, C_2, C_1/4, C_2/4]
params['mass_special'] = [1, 1, 1/4, 1/4]
params['v_c'] = x_c/t_c
params['k_BT'] = k_BT
params['U0'] = U0_1


### protocol setting

In [ ]:
zeroDissipation = False
# zeroDissipation = True

saveAllStates = True

params['sim_params'] = [_lambda, _theta, _eta]
params['capacitance'] = [C_1, C_2, C_1/4, C_2/4]
params['mass'] = [1, 1, 1/4, 1/4]
params['v_c'] = x_c/t_c
params['k_BT'] = k_BT
params['as_step'] = np.s_[::100] # the time step to skep for the all_state
params['as_step'] = np.s_[::] # the time step to skep for the all_state
params['percentage'] = 1 # For what percentage of the total sample do you want to keep in the output all_state

if zeroDissipation:
    params['sim_params'] = [_lambda * 0, _theta, _eta * 0]

params['circuit_parameters'] = {
    "C_factor":C_factor, "L_factor": L_factor, "I_m_factor": I_m_factor, "T": T, 
    "I_p_1": I_p_1, "I_p_2": I_p_2, "I_m_1": I_m_1, "I_m_2": I_m_2,
    "R_1": R_1, "R_2": R_2, "C_1": C_1, "C_2": C_2, "L_1": L_1, "L_2": L_2, 
    "characteristic_time": np.sqrt(C_1 * C_factor * L_1 * L_factor),
    "phi_1_x_on": phi_1_x_on_12, "phi_2_x_on": phi_2_x_on_12,
    "phi_1_dcx_on": phi_1_dcx_on_12, "phi_2_dcx_on": phi_2_dcx_on_12, "M_12_on": M_12_on,
    "gamma": gamma
}


"""
# step 2: Define initial condition and protocol
"""
manual_domain=[np.array([-5, -5]), np.array([5, 5])]



# µ = 0.06, φ2xdc = 1.79, φ1x = 0.59, and φ2x = 0.10.
    
initial_parameter_dict = {
        "U0_1": U0_1,     "U0_2": U0_2,     "gamma_1": gamma,  "gamma_2": gamma,
        "beta_1": beta_1,   "beta_2": beta_2,   "d_beta_1": d_beta_1 ,   "d_beta_2": d_beta_2,
        "phi_1_x": phi_1_x_off,  "phi_2_x": phi_2_x_off,  "phi_1_dcx": phi_1_dcx_off,  "phi_2_dcx": phi_1_dcx_off,
        "M_12": M_12_off, 'x_c': x_c
}



# bookmark
initial_parameter_dict["phi_1_dcx"], initial_parameter_dict["phi_2_dcx"], initial_parameter_dict["M_12"] = \
phi_1_dcx_off, phi_2_dcx_off, M_12_off


initial_parameter_dict["phi_1_dcx"] = phi_1_dcx_off
initial_parameter_dict["phi_2_dcx"] = phi_2_dcx_off
initial_parameter_dict["phi_1_x"]   = phi_1_x_off
initial_parameter_dict["phi_2_x"]   = phi_2_x_off
initial_parameter_dict["M_12"]      = M_12_off

m_12_factor = 1
percentage_factor = 0.99
# CE_1_a = {
#     "phi_1_x": phi_1_x_on_12, "phi_2_x": phi_2_x_on_12, "M_12": M_12_on, \
#     "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on_12, "name":"CE_1"
# }


# phi_2_x_on_12 = 0.1 
M_12_on = 0.06
phi_2_x_on_12 = 0.1

CE_1 = {
    "phi_1_x": phi_1_x_on_12, "phi_2_x": phi_2_x_on_12, "M_12": M_12_on, \
    "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on_12, "name":"CE_1"
}



In [ ]:
initial_parameter_dict["phi_1_x"]   = CE_1["phi_1_x"]
initial_parameter_dict["phi_2_x"]   = CE_1["phi_2_x"]
initial_parameter_dict["M_12"]      = CE_1["M_12"] 
initial_parameter_dict["phi_1_dcx"] = CE_1["phi_1_dcx"]
initial_parameter_dict["phi_2_dcx"] = CE_1["phi_2_dcx"] 

In [ ]:
"""
# step 1: Define potential
"""
coupled_fq_default_param = [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, x_c]
[phi_1_bound, phi_2_bound, phi_1dc_bound, phi_2dc_bound] = np.array([4, 4, 4, 4])/time_scale_factor
contour_range = [300, 2000]
    
coupled_fq_domain = [[-phi_1_bound, -phi_2_bound, -phi_1dc_bound, -phi_2dc_bound], \
                     [phi_1_bound, phi_2_bound, phi_1dc_bound, phi_2dc_bound]]

# coupled_fq_pot = Potential(coupled_flux_qubit_pot_with_offset_at_00_xy, coupled_flux_qubit_force, 14, 4,\
#                            default_params = coupled_fq_default_param,  relevant_domain = coupled_fq_domain)

coupled_fq_pot = Potential(coupled_flux_qubit_non_linear_approx_pot, coupled_flux_qubit_non_linear_approx_force, 14, 4,\
                           default_params = initial_parameter_dict,  relevant_domain = coupled_fq_domain)

# coupled_fq_pot = Potential(coupled_flux_qubit_non_linear_approx_pot_special, coupled_flux_qubit_non_linear_approx_force_special, 14, 4,\
                           # default_params = initial_parameter_dict,  relevant_domain = coupled_fq_domain)

### modify the protocol

In [ ]:

# longer CE
t_duration = 100



ratio_array = [0.998] * 5 # [phi_1x, phi_2x, M_12, phi_1xdc, phi_2xdc]

protocol_list_variable_duration = [
    # forward
    
    # create_CE_Protocol(50, CE_break_1),
    # create_CE_Protocol(50, CE_1, ratio_array),
    # create_CE_Protocol(20, CE_1),
    # create_CE_Protocol(200, CE_1, [1, 1, 1, 1, 1]),
    create_CE_Protocol(200, CE_1, [1, 10, 0, 1, 1]),
    # create_CE_Protocol(150, four_well)

]


new_protocol_list = [
    create_CE_Protocol(150, four_well)
]

protocol_list = protocol_list_variable_duration


# first initialization

### create cfqr object

In [ ]:
"""
# step 3: create the relevant storage protocol and computation protocol
"""
computation_protocol_parameter_dict = coupled_fq_protocol_library.customizedProtocol(initial_parameter_dict, \
                                                                    protocol_list)
storage_protocol, comp_protocol = create_system(computation_protocol_parameter_dict, modifiedFunction = None)

cfqr = cfq_runner.coupledFluxQubitRunner(potential = coupled_fq_pot, params = params, \
                                                storage_protocol= storage_protocol, \
                                                computation_protocol= comp_protocol, \
                                         protocol_list = protocol_list, \
                                        has_velocity=has_velocity)

init_state_used = np.load("default_init_state.npy")

protocol_time_array, protocol_time_index_array, protocol_all_time_array, protocol_time_all_index_array = cfqr.createProtocolTimeArray(protocol_list, params)
cfqr.initialize_sim()
# cfqr.set_sim_attributes(init_state=init_state_saved)
# cfqr.set_sim_attributes(init_state=init_state_used)


show parameter time graph and critical points vs t

In [ ]:
t_i, t_f = protocol_time_array
t_array = np.linspace(t_i, t_f, 4)
protocol_params_array = [cfqr.convert_params_list_into_dict(cfqr.protocol.get_params(_t)) for _t in t_array]


# target_parameter_array = ["phi_1_x", "phi_2_x", "M_12", "phi_1_dcx", "phi_2_dcx"]
# target_parameter = target_parameter_array[2]
time_and_target_protocol_params_array = [(_t, _p) for _t, _p in zip(t_array, protocol_params_array)]


In [ ]:
importlib.reload(coupled_fq_protocol_library)
fig, ax = plt.subplots(1, 2, figsize = [10, 5])

for _t, _p in time_and_target_protocol_params_array:
    print(_t, _p)
    target_parameter = ["M_12", "phi_2_x"]
    label_text = ", ".join([f"{_label} = {_p[_label]:.3g}" for _label in target_parameter])
    
    potential_plotline_figure_setting = {
        "horizontal_lim": [-4, 4], "vertical_lim": [200, 400], "ax": ax[0], "color": None, "label": label_text
    }
    slope_plotline_figure_setting = {
        "horizontal_lim": [-4, 4], "vertical_lim": [-2, 2], "ax": ax[1], "color": None, "label": label_text
    }
    coupled_fq_protocol_library.plot_potential_along_a_cutline(cfqr,\
    t = _t, cutlineValue = -2, plt_figure_setting = potential_plotline_figure_setting)
    
    coupled_fq_protocol_library.plot_slope_along_a_cutline(cfqr,\
    t = _t, cutlineValue = -2, plt_figure_setting = slope_plotline_figure_setting)



In [ ]:
# _stepped_time = protocol_all_time_array[::100]
# minimum_points_of_00 = [cfqr.get_minimum_at_t_for_a_set_of_parameter(_t, initial_parameter_dict, guess = [(-2, -2)])[0] for _t in _stepped_time]
# fig, ax = plt.subplots(1, 2, figsize = [8, 4])

# ax[0].scatter(_stepped_time, np.array(minimum_points_of_00)[:, 0])
# ax[0].set_xlabel(r't $(\tau_c)$', fontsize = 12)
# ax[0].set_ylabel(r'$\varphi_1$', fontsize = 12)

# ax[1].scatter(_stepped_time, np.array(minimum_points_of_00)[:, 1])
# ax[1].set_xlabel(r't $(\tau_c)$', fontsize = 12)
# ax[1].set_ylabel(r'$\varphi_2$', fontsize = 12)

# y_min, y_max = ax[1].set_ylim()
# for _t in protocol_time_array:
#     ax[1].vlines(x = _t, ymin = y_min, ymax = y_max, linestyle = "--")



# fig.tight_layout()



In [ ]:
importlib.reload(coupled_fq_protocol_library)

In [ ]:
def plot_contour_plot_and_cutline_plot(s_data, contour_ax = None, cutline_ax = None):
    """
    contour_ax = {"ax": None, "vmin": 0, "vmax": 1000}
    cutline_ax = {"ax": None, "min": 0, "max": 1000, "color": "red"}
    """
    if contour_ax:
        contour_ax['ax'].contourf(s_data['contourPlot']['X'], s_data['contourPlot']['Y'], s_data['contourPlot']['U'], 50, vmin = contour_ax["vmin"], vmax =contour_ax["vmax"], cmap = "afmhot")
        contour_axis_min, contour_axis_max = contour_ax['ax'].set_xlim()
        contour_ax['ax'].vlines(s_data['cutlinePlot']['cutlineValue'], ymin = contour_axis_min, ymax = contour_axis_max, color = cutline_ax['color'])

    if cutline_ax:
        cutline_ax['ax'].plot(s_data['cutlinePlot']['x_axis'], s_data['cutlinePlot']['y_axis'], color = cutline_ax['color'])
        cutline_ax['ax'].set_ylim(cutline_ax['min'], cutline_ax['max'])

In [ ]:
basic_external_params = {"phi_1_x": 0.59, "phi_2_x": 0,  "M_12": 0.06, "phi_1_dcx": 0.1, "phi_2_dcx": 1.79}

s = cfqr.create_params_dict_with_external_parameters(basic_external_params)


ratio = np.array([1.3, 0.3, 0.3, 0.3, 0.9])
updated_external_params = dict(zip(basic_external_params.keys(), np.array(list(basic_external_params.values())) * ratio))
s＿2 = cfqr.create_params_dict_with_external_parameters(updated_external_params)



In [ ]:
cutlineData={'cutlineValue': -2, 'cutlineDirection': 'v', 'color': 'red'}
s_data = coupled_fq_protocol_library.get_contour_and_cutline_data_with_given_params(cfqr, s, cutlineData)
s_2_data = coupled_fq_protocol_library.get_contour_and_cutline_data_with_given_params(cfqr, s_2, cutlineData)
difference = s_2_data['cutlinePlot']['y_axis'] - s_data['cutlinePlot']['y_axis']
fig, ax = plt.subplots(1, 2, figsize=[10, 4])
cutline_min, cutline_max = 0, 400

plot_contour_plot_and_cutline_plot(s_data, 
                                   contour_ax = {"ax": ax[0], "vmin": 0, "vmax": 600},  
                                   cutline_ax = {"ax": ax[1], "min": 200, "max": 600, "color": "red", "label": "s1"})
plot_contour_plot_and_cutline_plot(s_2_data, 
                                   cutline_ax = {"ax": ax[1], "min": 200, "max": 600, "color": "orange", "label": "s2"})

plt.show()

In [ ]:
cutlineData={'cutlineValue': 2, 'cutlineDirection': 'v', 'color': 'red'}
s_data = coupled_fq_protocol_library.get_contour_and_cutline_data_with_given_params(cfqr, s, cutlineData)

cutlineData['color']= "orange"
s_2_data = coupled_fq_protocol_library.get_contour_and_cutline_data_with_given_params(cfqr, s_2, cutlineData)
difference = s_2_data['cutlinePlot']['y_axis'] - s_data['cutlinePlot']['y_axis']
fig, ax = plt.subplots(1, 2, figsize=[10, 4])
cutline_min = 0
cutline_max = 600

plot_contour_plot_and_cutline_plot(s_data, 
                                   contour_ax = {"ax": ax[0], "vmin": 0, "vmax": 600},  
                                   cutline_ax = {"ax": ax[1], "min": cutline_min, "max": cutline_max, "color": "red"})
plot_contour_plot_and_cutline_plot(s_2_data, 
                                   cutline_ax = {"ax": ax[1], "min": cutline_min, "max": cutline_max, "color": "orange"})

plt.show()

### animation

In [ ]:
importlib.reload(coupled_fq_protocol_library)

In [ ]:
import sys, importlib
importlib.reload(sys.modules['edward_tools.coupled_fq_protocol_library'])
from edward_tools.jupyter_helper_function import work_analysis, KE_analysis, single_particle_analysis

def cutlineDataGeneratingFunction(initial_parameter_dict, direction = 'v', color = 'red', guess = [(-2, -2)]):
    coord_index = 0 if direction == "v" else 1
    def tracking_minimum_point(_t):
        return  [cfqr.get_minimum_at_t_for_a_set_of_parameter(_t, initial_parameter_dict, guess = guess)[0][coord_index], direction, color]
    return tracking_minimum_point

### without cutlineGeneratingFunction

In [ ]:
import matplotlib.animation as animation

# animation_play = True
# if animation_play:
frame_skip = int(1/50 * 1/params['dt'])
# print("geenerated at ", datetime.datetime.now())

contourData = {
    "vmax": 0, "vmin": 500,
    "manual_domain": [np.array([-5, -5]), np.array([5, 5])],
    "contour_range": [0, 400],
    "title": f"T = {T}K, L = {L_1 * 1e12}pH"
}

cutlineInformation = {
    "cutlineList": [(-2, "v","red")],
    "cutlineGeneratingFunction": None,
    "cutlineXLimit": [-4,4],
    "cutlineYLimit": [50, 400]
}

particleInformation = {
    "showParticles": False,
    "project_item": ["00", "01"],
    "particle_opacity": 0.5,
    "pColor": {"00": "#061DF7", "01": "red", "10": "#3FC7F2", "11": "#F187F4"}
}

animation_setting = {
    "frame_skip": 10,
    "save_path": None, 
    "save_dict": None,
    "interval": 100,
    "blit": False
}

protocol_graph_setting = {
    "key": ["phi_1_x", "phi_2_x", "phi_1_dcx", "phi_2_dcx", "M_12"]

}


fig, ax = plt.subplots(1, 3, figsize=[15, 5])

targetTimeArray = list(range(protocol_time_array[0], protocol_time_array[1]+1))

ani, fig, ax = coupled_fq_protocol_library.animate_sim_flux_qubit_with_cutline_and_projection(cfqr,
    time_array=targetTimeArray, params = params, legend=True, 
    plot_axis = [0, 1], slice_values = None, fig_ax=(fig, ax), animation_setting= animation_setting,
    contourData = contourData, cutlineInformation = cutlineInformation, particleInformation = particleInformation,
    protocol_graph_setting = protocol_graph_setting, ax0_title = None, offset_potential = False)



fig.tight_layout()
writer = animation.PillowWriter(fps=5, metadata=dict(artist='Me'), bitrate=1800)
ani.save('scatter.gif', writer=writer)
plt.close()
# HTML(ani.to_jshtml(fps=10))

In [ ]:
Image(filename="scatter.gif")

In [ ]:
import matplotlib.animation as animation

# animation_play = True
# if animation_play:
frame_skip = int(1/50 * 1/params['dt'])
# print("geenerated at ", datetime.datetime.now())

contourData = {
    "vmax": 0, "vmin": 500,
    "manual_domain": [np.array([-5, -5]), np.array([5, 5])],
    "contour_range": [0, 400],
    "title": f"T = {T}K, L = {L_1 * 1e12}pH"
}

cutlineInformation = {
    "cutlineList": [(2, "v","red")],
    "cutlineGeneratingFunction": None,
    "cutlineXLimit": [-4,4],
    "cutlineYLimit": [50, 400]
}

particleInformation = {
    "showParticles": False,
    "project_item": ["10", "11"],
    "particle_opacity": 0.5,
    "pColor": {"00": "#061DF7", "01": "red", "10": "#3FC7F2", "11": "#F187F4"}
}

animation_setting = {
    "frame_skip": 10,
    "save_path": None, 
    "save_dict": None,
    "interval": 100
}


targetTimeArray = list(range(protocol_time_array[0], protocol_time_array[1]+1))

ani, fig, ax = coupled_fq_protocol_library.animate_sim_flux_qubit_with_cutline_and_projection(cfqr, frame_skip=30, time_array=targetTimeArray, params = params, legend=True, 
    plot_axis = [0, 1], slice_values = None, fig_ax=None, 
    contourData = contourData, cutlineInformation = cutlineInformation, particleInformation = particleInformation,
    save_path = None, save_dict = None, ax0_title = None, offset_potential = False)



fig.tight_layout()
writer = animation.PillowWriter(fps=10, metadata=dict(artist='Me'), bitrate=1800)
ani.save('scatter.gif', writer=writer)
plt.close()
# HTML(ani.to_jshtml(fps=10))

In [ ]:
Image(filename="scatter.gif")

### with cutlineGeneratingFunction

In [ ]:
animation_play = True
# animation_play = False
if animation_play:

    frame_skip = int(1/50 * 1/params['dt'])
    print("geenerated at ", datetime.datetime.now())
    targetTimeArray = protocol_all_time_array[simResult['cfqr'].sim.target_step_index]
    ani, fig, ax = coupled_fq_protocol_library.animate_sim_flux_qubit_with_cutline_and_projection(simResult['cfqr'], time_array=targetTimeArray, frame_skip = frame_skip, plot_axis= [0,1],
                                                                                   vmin=graph_setting[T]['vmin'], vmax=graph_setting[T]['vmax'], ax0_title = f"T = {T}K, L = {L_1 * 1e12}pH",
                                                                                   params=params, cutlineGeneratingFunction = cutlineDataGeneratingFunction(initial_parameter_dict),
                                                                                   pColor=pColor, contour_range = [0, 500], cutlineXLimit=[-4, 4], cutlineYLimit=[0, 500])
    fig.tight_layout()
    writer = animation.PillowWriter(fps=15, metadata=dict(artist='Me'), bitrate=1800)
    ani.save('scatter.gif', writer=writer)
    Image(filename="scatter.gif")

### save data analysis

In [ ]:
print(f"current comment: {params['comment']}")
response = input("Are you happy with the current comment? If not, give me a new comment.")
if response == "y":
    pass
else:
    params['comment'] = response

In [ ]:
cfq_batch_sweep.saveSimulationResult(simResult, U0_1, timeOrStep = 'step', save = True, save_final_state = False, saveFolderPath = "coupled_flux_qubit_protocol/coupled_flux_qubit_data_gallery", save_all_state = False)

In [ ]:
beta_array = np.linspace(2, 3, 5)

In [ ]:
target_value = 2.3 * np.cos(1.79/2)

In [ ]:
target_value_array = [2 * np.arccos(target_value/_beta) for _beta in beta_array]

In [ ]:
target_value_array

In [ ]:
# (t, work, std)
work_cost_of_CT = [(100, 9.64, 0.0437), (150, 5.82, 0.0347), (250, 3.49± 0.0327)]

In [ ]:
phi_1 = 2.3
phi_1x = 
phi_2x = 
phi_2xdc = 
M_12 = 0.05
a = 1.25
beta = 2.3
circuit_params = {
    "L": 25-12, "T": 4.2,
    "beta": 2.3, "d_beta": 0,
    "phi_1x": 0.59,   "phi_2x": 0.087, "phi_1xdc": 0, "phi_2xdc": 1.925, "M_12": 0.05
}

def pot_function(circuit_params=circuit_params, detail_info = False):
    U_ratio = 1

    beta_1 = circuit_params["beta"]
    beta_2 = circuit_params["beta"]
    d_beta_1 = circuit_params["d_beta"]
    d_beta_2 = circuit_params["d_beta"]
    _phi_1x = circuit_params["phi_1x"]
    _phi_2x = circuit_params["phi_2x"]
    _phi_1xdc = circuit_params["phi_1xdc"]
    _phi_2xdc = circuit_params["phi_2xdc"]
    _M_12 = circuit_params["M_12"]
    _xi = 1
    _phi_1dc = _phi_1xdc
    _phi_2dc = _phi_2xdc

    def Fcn(coord):
        _phi_1, _phi_2 = coord
        u1_1 = 1/2 * _xi * (_phi_1 - _phi_1x)**2
        u1_3 = beta_1 * np.cos(_phi_1) * np.cos(_phi_1dc/2)
        u1_4 = -d_beta_1 * np.sin(_phi_1) * np.sin(_phi_1dc/2)

        u2_1 = 1/2 * _xi * (_phi_2 - _phi_2x)**2
        u2_3 = beta_2 * np.cos(_phi_2) * np.cos(_phi_2dc/2)
        u2_4 = -d_beta_2 * np.sin(_phi_2) * np.sin(_phi_2dc/2)
        u3 = _M_12 * _xi * (_phi_1 - _phi_1x) * (_phi_2 - _phi_2x)

        if detail_info:
            return {
                "u1_1": u1_1, "u1_3": u1_3, "u1_4": u1_4,
                "u2_1": u2_1, "u2_3": u2_3, "u1_4": u2_4,
                "u3": u3
            }
        else:
            return U_ratio * (u1_1 + u1_3 + u1_4 + u2_1 + u2_3 + u2_4 + u3)

    return Fcn

In [ ]:
delta_U_11

In [ ]:
circuit_params = {

def pot_function(circuit_params=circuit_params, detail_info = False):
    # L = circuit_params["L"]
    # T = circuit_params["T"]
    # k_BT = k_B * T
    # x_c = PHI_0/(2 * np.pi)
    # U_ratio = x_c**2 / (k_BT * L)
    U_ratio = 1

    beta_1 = circuit_params["beta"]
    beta_2 = circuit_params["beta"]
    d_beta_1 = circuit_params["d_beta"]
    d_beta_2 = circuit_params["d_beta"]
    _phi_1x = circuit_params["phi_1x"]
    _phi_2x = circuit_params["phi_2x"]
    _phi_1xdc = circuit_params["phi_1xdc"]
    _phi_2xdc = circuit_params["phi_2xdc"]
    _M_12 = circuit_params["M_12"]
    _xi = 1
    _phi_1dc = _phi_1xdc
    _phi_2dc = _phi_2xdc

    def Fcn(coord):
        _phi_1, _phi_2 = coord
        u1_1 = 1/2 * _xi * (_phi_1 - _phi_1x)**2
        u1_3 = beta_1 * np.cos(_phi_1) * np.cos(_phi_1dc/2)
        u1_4 = -d_beta_1 * np.sin(_phi_1) * np.sin(_phi_1dc/2)

        u2_1 = 1/2 * _xi * (_phi_2 - _phi_2x)**2
        u2_3 = beta_2 * np.cos(_phi_2) * np.cos(_phi_2dc/2)
        u2_4 = -d_beta_2 * np.sin(_phi_2) * np.sin(_phi_2dc/2)
        u3 = _M_12 * _xi * (_phi_1 - _phi_1x) * (_phi_2 - _phi_2x)

        if detail_info:
            return {
                "u1_1": u1_1, "u1_3": u1_3, "u1_4": u1_4,
                "u2_1": u2_1, "u2_3": u2_3, "u1_4": u2_4,
                "u3": u3
            }
        else:
            return U_ratio * (u1_1 + u1_3 + u1_4 + u2_1 + u2_3 + u2_4 + u3)

    return Fcn

In [ ]:
_phi_1xdc_array

In [ ]:


def get_barrier_height(_phi_1xdc_array, protocol_2, protocol_duration, N):
    result_array = []

    for _phi_1xdc in _phi_1xdc_array:
        protocol_2["phi_1xdc"] = _phi_1xdc
        protocol_2["beta"] = beta_1
        protocol_2["d_beta"] = 0
        critical_dict = find_all_critical_points_for_all_potential(protocol_2, guess = [(-2, -2), (-2, 2), (2, -2), (2, 2), (-2,0), (0, -2), (2, 0), (0, 2)])
        critical_points = list(critical_dict.values())[0]

        critical_potential = [pot_function(protocol_2)([x, y]) for x, y in critical_points]
        
        delta_U17 = critical_potential[7] - critical_potential[1]
        delta_U37 = critical_potential[7] - critical_potential[3]

        result_array.append({
            "coord": critical_points, "delta_U17": delta_U17 * U0_1, "delta_U37": delta_U37 * U0_1, "phi_1xdc": _phi_1xdc
        })

    escape_rate_info_array = []
    for item in result_array:
        # second_derivative_array = np.array([cal_second_derivative(_phi_1, _phi_1xdc) for _phi_1, _phi_1xdc in zip(phi_1, phi_1xdc)])
        target_critial_point_info = item['coord'][1][0], item['phi_1xdc'], item['delta_U17']
        second_derivative_array = np.array(cal_second_derivative(target_critial_point_info[0], target_critial_point_info[1]))
        plasma_freq = np.sqrt(second_derivative_array / (L * C))
        escape_rate = plasma_freq / (2 * np.pi) * np.exp(-target_critial_point_info[2])
        escape_event_per_operation = escape_rate * protocol_duration * N
        dwell_time = 1 / escape_rate
        escape_rate_info_array.append([item['delta_U17'], escape_rate, escape_event_per_operation])

    return {"critical_info": result_array, "escape_rate_info_array": escape_rate_info_array}


In [ ]:
mu_factor_1 = -0.5
_phi_1xdc_array = np.linspace(1.90, 2.2, 40)
corrected_phi_1dc_array = []
_beta, _gamma = 2.3, 9

for _phi_1xdc in _phi_1xdc_array:
    Fcn = setup_transcendtal_equation(_phi_1xdc, _beta, _gamma)
    sol = fsolve(Fcn, _phi_1xdc)[0]
    corrected_phi_1dc_array.append(sol)


protocol_duration = np.sqrt(L * C) * 50
N = 124_000
protocol_2 = {"phi_1x": 0.095 * mu_factor_1, "phi_2x": 0, "mu_12": 0.0475 * mu_factor_1, "phi_1xdc": _phi_1xdc, "phi_2xdc": 0, "name": "acceleration", "color": "g", "linestyle": "-"}


barrier_height_info = get_barrier_height(_phi_1xdc_array, protocol_2, protocol_duration, N/2)
corrected_barrier_height_info = get_barrier_height(corrected_phi_1dc_array, protocol_2, protocol_duration, N/2)


## convert

In [ ]:
def setup_transcendtal_equation(phi_1xdc, beta, gamma):
    def Fcn(phi_1dc):
        return phi_1dc - beta / (2 * gamma) * np.sin(phi_1dc/2) - phi_1xdc
    return Fcn


def convert_phi_xdc_from_beta(beta_1, phi_1xdc_1, beta_2, gamma = 9):
    def setup_Fcn_dc_to_xdc(beta, gamma, phi_1dc):
        def Fcn_dc_to_xdc(phi_1xdc):
            return phi_1dc - beta / (2 * gamma) * np.sin(phi_1dc/2) - phi_1xdc
        return Fcn_dc_to_xdc
    
    def setup_Fcn_xdc_to_dc(beta, gamma, phi_1xdc):
        def Fcn_xdc_to_dc(phi_1dc):
            return phi_1dc - beta / (2 * gamma) * np.sin(phi_1dc/2) - phi_1xdc
        return Fcn_xdc_to_dc

    phi_1dc = fsolve(setup_Fcn_xdc_to_dc(beta_1, gamma, phi_1xdc_1), phi_1xdc_1)[0]
    print(f"in beta_1 system: {phi_1dc}, {phi_1xdc_1}")

    phi_1dc_p = np.arccos(beta_1 *  np.cos(phi_1dc/2) / beta_2) * 2
    phi_1xdc_p = fsolve(setup_Fcn_dc_to_xdc(beta_2, gamma, phi_1dc_p), phi_1dc_p)[0]
    print(f"in beta_2 system: {phi_1dc_p}, {phi_1xdc_p}")


In [ ]:
convert_phi_xdc_from_beta(1.35, 1.7, 2.3)

In [ ]:
delta_U17_array = [x[0] for x in barrier_height_info['escape_rate_info_array']]
escape_event_per_op_array = [x[2] for x in barrier_height_info['escape_rate_info_array']]
corrected_delta_U17_array = [x[0] for x in corrected_barrier_height_info['escape_rate_info_array']]
corrected_escape_event_per_op_array = [x[2] for x in corrected_barrier_height_info['escape_rate_info_array']]


In [ ]:
plt.plot(corrected_phi_1dc_array, corrected_delta_U17_array)

In [ ]:


# Experiment 2 (2025-07-15) (3rd data, i.e. E_B = 25.0), Experiment 2 (2025-07-12)
simulation_data_barrier = [11.3, 21.0, 25.0, 30, 40.2, 50]
simulation_data_escape_event = [31193, 1112, 60, 0, 0, 0]
simulated_work = [24.4, 22.7, 25.4, 29.4, 34.6, 55]
simulated_work_error = np.array([0.225, 0.228, 0.324, 0.366, 0.463, 0.742]) * 3



In [ ]:
import matplotlib as mpl
rc_dict = {'font.size':16, 'axes.labelsize':'large', 'ytick.right':False,'legend.loc':'upper right', 'legend.fontsize':'xx-small', 'figure.autolayout':True, 'figure.figsize': (10,10), 'mathtext.fontset':'stix', 'font.family':'STIXGeneral'}
mpl.rcParams.update(rc_dict)

escape_event_per_op_array = [x[2] for x in barrier_height_info['escape_rate_info_array']]

fig, ax = plt.subplots(1, 1, figsize = [5, 5])
ax.scatter(delta_U17_array, escape_event_per_op_array, label = "naive")
ax.scatter(delta_U17_array, corrected_escape_event_per_op_array, label = "corrected")
ax.scatter(simulation_data_barrier, simulation_data_escape_event, label = "simulation")
ax.hlines(y = 1, xmin = 0, xmax = 50, linestyles="--")
ax.set_yscale('log')
ax.set_xlabel(r"intended barrier height ($k_BT$)")
ax.set_ylabel("escape events per operation")
ax.set_xlim(9.5,51 )
plt.legend(fontsize=13)

In [ ]:
# escape_rate_info_array = np.array(escape_rate_info_array)
# corrected_escape_rate_info_array = np.array(corrected_escape_rate_info_array)

# barrier_height = escape_rate_info_array[:, 0]
# corrected_barrier_height = corrected_escape_rate_info_array[:, 0]
# expected_escape_number = escape_rate_info_array[:, 1] * protocol_duration * N/2
# corrected_expected_escape_number = corrected_escape_rate_info_array[:, 1] * protocol_duration * N/2

In [ ]:
expected_escape_number

In [ ]:
# plt.scatter(simulation_data_barrier, simulation_data_escape_event, label = "simulation", color = "blue")
plt.scatter(barrier_height, expected_escape_number, label = "theory", color = "orange")
plt.xlabel("$\Delta E_B (k_BT)$")
plt.ylabel(r"escape events per operation")
plt.yscale("log")
plt.title(r"t = 50 $t_c$")
xmin, xmax = plt.xlim()
# plt.hlines(y = 1e-6, xmin = xmin, xmax = xmax, linestyles = "--")
plt.legend()

In [ ]:
plt.scatter(simulation_data_barrier, simulation_data_escape_event, label = "simulation", color = "blue")
plt.scatter(barrier_height, expected_escape_number, label = "theory", color = "orange")
plt.scatter(corrected_barrier_height, corrected_expected_escape_number, label = "corrected", color = "red")
plt.xlabel("$\Delta E_B (k_BT)$")
plt.ylabel(r"escape events per operation")
plt.yscale("log")
plt.title(r"t = 50 $t_c$")
xmin, xmax = plt.xlim()
plt.hlines(y = 1, xmin = xmin, xmax = xmax, linestyles = "--")
plt.legend()

In [ ]:
plt.errorbar(x = simulation_data_barrier, y = simulated_work, yerr = simulated_work_error, label = "simulation", fmt = "o")
plt.xlabel("∆E_B (k_BT)")
plt.ylabel("work")
plt.title("work done vs ∆E_B")
plt.legend()

In [ ]:
protocol_duration

In [ ]:
plt.scatter(barrier_height, 1/escape_rate_info_array[:,1] / protocol_duration)
plt.xlabel("∆E_B (k_BT)")
plt.ylabel("robustness")
plt.title("t = 80 t_c, L = 5pH, C = 500fF")
plt.yscale("log")

In [ ]:
k_BT

In [ ]:
expected_escape_number

In [ ]:
np.sqrt(L * C) 

# calculate escape rate

In [ ]:





def second_derivative_of_pot_function(protocol_2, beta, d_beta, gamma):
    beta_1 = beta
    beta_2 = beta
    d_beta_1 = d_beta
    d_beta_2 = d_beta

    _phi_1xdc = protocol_2["phi_1xdc"]
    _phi_2xdc = protocol_2["phi_2xdc"]

    def Fcn(x):
        return [
            1 - beta_1 * np.cos(x[0]) * np.cos(_phi_1xdc/2) - d_beta_1 * np.sin(x[0]) * np.sin(_phi_1xdc/2),
            1 - beta_2 * np.cos(x[1]) * np.cos(_phi_2xdc/2) - d_beta_2 * np.sin(x[1]) * np.sin(_phi_2xdc/2)
        ]
    return Fcn



def find_all_minimum_points_for_all_potential(circuit_params, guess = [(0, 0)]):
    """"
    To find all the maximum points, of the potential function
    """
    solution_set = [optimize.fmin(detailed_pot_function(circuit_params), _g, disp=False) for _g in guess]
    # energy_set = [coupled_flux_qubit_non_linear_approx_pot(sol[0], sol[1], _phi_1dcx, _phi_2dcx, _params_at_t) for sol in solution_set]
    return {"coord": solution_set}





In [ ]:
mu_factor_h2 = 0.40
beta_1 = 1.35

if beta_1 == 2.3:
    mu_12_factor_beta = 0.05
if beta_1 == 1.35:
    mu_12_factor_beta = 0.0765
protocol_harmonic_h2 = {"phi_1x": 0.1 * mu_factor_h2, "phi_2x": 0.0, "mu_12": mu_12_factor_beta * mu_factor_h2, "phi_1xdc": 1.1,  "phi_2xdc": 0, "name": "h2", "color": "g", "linestyle": "-"} 
get_barrier_height(protocol_harmonic_h2, beta_1,  gamma = 16, protocol_duration= 16, N = 1e12/2)

# speed comparison

In [ ]:
import matplotlib.pyplot as plt
import numpy as np


def plot_graph(KE_data_file_name, ax_flatten, label, linestyle = "-", showProtocolTime = False, showError = False, color = "blue"):
    KE_info = np.load(KE_data_file_name, allow_pickle=True).item()
    KE_data = KE_info["KE_data"]
    KE_mean = KE_data[0, ...]
    KE_std = KE_data[1, ...]

    protocol_time_array = KE_info["protocol_time_array"]
    normalized_protocol_time_array = protocol_time_array / max(protocol_time_array)

    normalized_time = np.array(range(len(KE_mean[0, :])))
    normalized_time = normalized_time / np.max(normalized_time)

    ax_flatten[2].plot(normalized_time, KE_mean[0, :], label = label, linestyle = linestyle, color = color)
    ax_flatten[0].plot(normalized_time, KE_mean[1, :], label = label, linestyle = linestyle, color = color)
    ax_flatten[3].plot(normalized_time, KE_mean[2, :], label = label, linestyle = linestyle, color = color)
    ax_flatten[1].plot(normalized_time, KE_mean[3, :], label = label, linestyle = linestyle, color = color)

    if showError:
        ax_flatten[2].errorbar(normalized_time[::20], KE_mean[0, ::20], yerr = KE_std[0, ::20], linestyle='', color = color)
        ax_flatten[0].errorbar(normalized_time[::20], KE_mean[1, ::20], yerr = KE_std[1, ::20], linestyle='', color = color)
        ax_flatten[3].errorbar(normalized_time[::20], KE_mean[2, ::20], yerr = KE_std[2, ::20], linestyle='', color = color)
        ax_flatten[1].errorbar(normalized_time[::20], KE_mean[3, ::20], yerr = KE_std[3, ::20], linestyle='', color = color)


    ymin, ymax = 0, 0
    if showProtocolTime:
        for ax in ax_flatten:
            for i, x in enumerate(normalized_protocol_time_array):
                if i == 0:
                    ymin, ymax = ax.get_ylim()
                
                ax.vlines(x, ymin = ymin, ymax = ymax, color = "grey", alpha = 0.15, linestyle = "--")

    ax_flatten[0].legend()

In [ ]:
fig, ax = plt.subplots(2, 2, figsize = [10, 10])
ax_flatten = ax.flatten()

plot_graph(os.path.join("KE_data", KE_data_file_name), ax_flatten, label = "t = 42", linestyle= "--", showProtocolTime = True, showError=True)

In [ ]:
def phi_analysis_of_individual_particles(self, target_axis = "phi_1"):
    mapping = {"phi_1": 0, "phi_2": 1, "phi_1dc": 2, "phi_2dc": 3}
    target_axis_index = mapping[target_axis]

    pIndex = self.getIndexOfParticles()
    """To investigate the average KE of each type of particles"""
    all_state = self.get_all_state()

    index_of_00 = pIndex["00"]
    index_of_01 = pIndex["01"]
    index_of_10 = pIndex["10"]
    index_of_11 = pIndex["11"]

    phi_00 = all_state[index_of_00, ..., target_axis_index, 0]
    phi_01 = all_state[index_of_01, ..., target_axis_index, 0]
    phi_10 = all_state[index_of_10, ..., target_axis_index, 0]
    phi_11 = all_state[index_of_11, ..., target_axis_index, 0]

    phi_mean_array_00, phi_std_array_00 = np.mean(phi_00, axis = 0), np.std(phi_00, axis = 0) * 3
    phi_mean_array_01, phi_std_array_01 = np.mean(phi_01, axis = 0), np.std(phi_01, axis = 0) * 3
    phi_mean_array_10, phi_std_array_10 = np.mean(phi_10, axis = 0), np.std(phi_10, axis = 0) * 3
    phi_mean_array_11, phi_std_array_11 = np.mean(phi_11, axis = 0), np.std(phi_11, axis = 0) * 3
    
    print(len(phi_mean_array_00), len(phi_std_array_00))

    item = np.array([
        [phi_mean_array_00, phi_mean_array_01, phi_mean_array_10, phi_mean_array_11],
        [phi_std_array_00, phi_std_array_01, phi_std_array_10, phi_std_array_11]
    ])


    return item

def plot_graph(phi_data, KE_info, ax_flatten, label, linestyle = "-", showProtocolTime = False, showError = False, color = "blue"):
    KE_data = KE_info["KE_data"]
    KE_mean = KE_data[0, ...]
    KE_std = KE_data[1, ...]

    phi_mean = phi_data[0, ...]
    phi_std = phi_data[1, ...]

    protocol_time_array = KE_info["protocol_time_array"]
    normalized_protocol_time_array = protocol_time_array / max(protocol_time_array)

    normalized_time = np.array(range(len(KE_mean[0, :])))
    normalized_time = normalized_time / np.max(normalized_time)

    ax_flatten[2].plot(normalized_time, KE_mean[0, :], label = label, linestyle = linestyle, color = color)
    ax_flatten[0].plot(normalized_time, KE_mean[1, :], label = label, linestyle = linestyle, color = color)
    ax_flatten[3].plot(normalized_time, KE_mean[2, :], label = label, linestyle = linestyle, color = color)
    ax_flatten[1].plot(normalized_time, KE_mean[3, :], label = label, linestyle = linestyle, color = color)


    ax2, ax0, ax3, ax1 = ax_flatten[2].twinx(), ax_flatten[0].twinx(), ax_flatten[3].twinx(), ax_flatten[1].twinx()
    ax2.plot(normalized_time, phi_mean[0, :], label = label, linestyle = linestyle, color = "red")
    ax0.plot(normalized_time, phi_mean[1, :], label = label, linestyle = linestyle, color = "red")
    ax3.plot(normalized_time, phi_mean[2, :], label = label, linestyle = linestyle, color = "red")
    ax1.plot(normalized_time, phi_mean[3, :], label = label, linestyle = linestyle, color = "red")


    if showError:
        ax_flatten[2].errorbar(normalized_time[::20], KE_mean[0, ::20], yerr = KE_std[0, ::20], linestyle='', color = color)
        ax_flatten[0].errorbar(normalized_time[::20], KE_mean[1, ::20], yerr = KE_std[1, ::20], linestyle='', color = color)
        ax_flatten[3].errorbar(normalized_time[::20], KE_mean[2, ::20], yerr = KE_std[2, ::20], linestyle='', color = color)
        ax_flatten[1].errorbar(normalized_time[::20], KE_mean[3, ::20], yerr = KE_std[3, ::20], linestyle='', color = color)

        ax2.errorbar(normalized_time[::20], phi_mean[0, ::20], yerr = phi_std[0, ::20], linestyle='', color = "red")
        ax0.errorbar(normalized_time[::20], phi_mean[1, ::20], yerr = phi_std[1, ::20], linestyle='', color = "red")
        ax3.errorbar(normalized_time[::20], phi_mean[2, ::20], yerr = phi_std[2, ::20], linestyle='', color = "red")
        ax1.errorbar(normalized_time[::20], phi_mean[3, ::20], yerr = phi_std[3, ::20], linestyle='', color = "red")

    ymin, ymax = 0, 0
    if showProtocolTime:
        for ax in ax_flatten:
            for i, x in enumerate(normalized_protocol_time_array):
                if i == 0:
                    ymin, ymax = ax.get_ylim()
                
                ax.vlines(x, ymin = ymin, ymax = ymax, color = "grey", alpha = 0.15, linestyle = "--")

    ax_flatten[0].legend()


In [ ]:
KE_position_data = {
    "KE_data": simResult['cfqr'].KE_analysis_of_individual_particles(),
    "protocol_time_array": np.array(simResult['cfqr'].protocol_time_array),
    "protocol_list": protocol_list
}


KE_data_file_name = "Experiment_1_2025_08_31_KE_data.npy"

fig, ax = plt.subplots(2, 2, figsize = [10, 10])
ax_flatten = ax.flatten()
phi_data = phi_analysis_of_individual_particles(simResult['cfqr'])
plot_graph(phi_data, KE_data, ax_flatten, label = "t = 42", linestyle= "--", showProtocolTime = True, showError=True)

save_KE_data = True
save_KE_data = False
if save_KE_data:
    np.save(os.path.join("KE_data", KE_data_file_name), KE_data, allow_pickle=True)




In [ ]:

phi_mean = phi_data[0, ...]
phi_std = phi_data[1, ...]
label = "t = 42"
linestyle = "--"
color = "blue"
showError = True
fig, ax = plt.subplots(2, 2, figsize = [10, 10])
ax_flatten = ax.flatten()

# print(phi_mean[0, ...])

normalized_time = np.array(range(len(phi_mean[0, ...])))
normalized_time = normalized_time / np.max(normalized_time)

ax_flatten[2].plot(normalized_time, phi_mean[0, :], label = label, linestyle = linestyle, color = color)
ax_flatten[0].plot(normalized_time, phi_mean[1, :], label = label, linestyle = linestyle, color = color)
ax_flatten[3].plot(normalized_time, phi_mean[2, :], label = label, linestyle = linestyle, color = color)
ax_flatten[1].plot(normalized_time, phi_mean[3, :], label = label, linestyle = linestyle, color = color)

# if showError:
#     ax_flatten[2].errorbar(normalized_time[::20], phi_mean[0, ::20], yerr = phi_mean[0, ::20], linestyle='', color = color)
#     ax_flatten[0].errorbar(normalized_time[::20], phi_mean[1, ::20], yerr = phi_mean[1, ::20], linestyle='', color = color)
#     ax_flatten[3].errorbar(normalized_time[::20], phi_mean[2, ::20], yerr = phi_mean[2, ::20], linestyle='', color = color)
#     ax_flatten[1].errorbar(normalized_time[::20], phi_mean[3, ::20], yerr = phi_mean[3, ::20], linestyle='', color = color)


# ax_flatten[0].legend()



# # plot_graph(KE_data, ax_flatten, label = "t = 42", linestyle= "--", showProtocolTime = True, showError=True)